This notebook was used to develop the ETL pipeline for getting new COVID-19 data from Johns Hopkins via Github and to produce a clean dataframe.  In the process of producing the clean dataframe, it also loads cleaned dataframes for many regional populations so per capita calculations can be included in the final COVID dataframe.  It finally saves the clean dataframe as a pickle file on my local drive and uploads a copy to my Google Drive.  

In [237]:
import time
t0 = time.time()
from github.MainClass import Github

# read my github access token from my secrets folder
# a new token can be created here: https://github.com/settings/tokens
with open("C:/Users/adiad/Anaconda3/envs/CovidApp36/covidapp/secret_credentials/github_token.txt", "r") as file:
    token = file.read()

g = Github(token)
repo = g.get_repo("CSSEGISandData/COVID-19")
file_list = repo.get_contents("csse_covid_19_data/csse_covid_19_daily_reports")

In [238]:
import pandas as pd
import numpy as np
import plotly, plotly.graph_objects as go
import matplotlib, matplotlib.pyplot as plt, matplotlib.cm as cm

github_dir_path = 'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports/'
file_path = github_dir_path  + str(file_list[1]).split('/')[-1].split(".")[0]+ '.csv'

file_date = str(file_list[1]).split('/')[-1].split(".")[0]
file_path = github_dir_path + file_date + '.csv'
df = pd.read_csv(file_path, error_bad_lines=False)
df["Date"] = file_date

for file in file_list[2:-1]:
  file_date = str(file).split('/')[-1].split(".")[0]
  file_path = github_dir_path  + file_date + '.csv'
  file_df = pd.read_csv(file_path, error_bad_lines=False)
  file_df["Date"] = file_date

  # rename columns which have changed from the orignal names in recent files
  if any(file_df.columns.isin(["Country_Region"])):
    file_df = file_df.rename(columns={"Country_Region": "Country/Region"})
  if any(file_df.columns.isin(["Province_State"])):
    file_df = file_df.rename(columns={"Province_State": "Province/State"})
  if any(file_df.columns.isin(["Last_Update"])):
    file_df = file_df.rename(columns={"Last_Update": "Last Update"})
  if any(file_df.columns.isin(["Long_"])):
    file_df = file_df.rename(columns={"Long_": "Longitude"})
  if any(file_df.columns.isin(["Lat"])):
    file_df = file_df.rename(columns={"Lat": "Latitude"})
  if any(file_df.columns.isin(["Admin2"])):
    file_df = file_df.rename(columns={"Admin2": "County"})
  
  df = df.append(file_df, ignore_index=True, sort=False)

df.head(20)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,Anhui,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Beijing,Mainland China,1/22/2020 17:00,14.0,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chongqing,Mainland China,1/22/2020 17:00,6.0,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fujian,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gansu,Mainland China,1/22/2020 17:00,NaN,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Guangdong,Mainland China,1/22/2020 17:00,26.0,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Guangxi,Mainland China,1/22/2020 17:00,2.0,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Guizhou,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Hainan,Mainland China,1/22/2020 17:00,4.0,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Hebei,Mainland China,1/22/2020 17:00,1.0,NaN,NaN,01-22-2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [239]:
# format columns
df["Country/Region"] = df["Country/Region"].astype(str)
df["Province/State"] = df["Province/State"].astype(str)
df["County"] = df["County"].astype(str)
df.Date = pd.to_datetime(df.Date)
df["Last Update"] = pd.to_datetime(df["Last Update"])

In [240]:
# the following rows appear to give bogus duplicate data, so they will be deleted
df[df["Country/Region"].isin(['The Bahamas', 'Bahamas, The'])]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
6152,nan,The Bahamas,2020-03-16 03:33:03,1.0,0.0,0.0,2020-03-16,24.2500,-76.0000,NaN,nan,NaN,NaN,NaN,NaN
6427,nan,The Bahamas,2020-03-16 03:33:03,1.0,0.0,0.0,2020-03-17,24.2500,-76.0000,NaN,nan,NaN,NaN,NaN,NaN
6705,nan,The Bahamas,2020-03-16 03:33:03,1.0,0.0,0.0,2020-03-18,24.2500,-76.0000,NaN,nan,NaN,NaN,NaN,NaN
6953,nan,"Bahamas, The",2020-03-19 11:33:39,3.0,0.0,0.0,2020-03-19,25.0343,-77.3963,NaN,nan,NaN,NaN,NaN,NaN
7012,nan,The Bahamas,2020-03-19 12:13:38,0.0,0.0,0.0,2020-03-19,24.2500,-76.0000,NaN,nan,NaN,NaN,NaN,NaN
7254,nan,"Bahamas, The",2020-03-19 11:33:39,3.0,0.0,0.0,2020-03-20,25.0343,-77.3963,NaN,nan,NaN,NaN,NaN,NaN
7311,nan,The Bahamas,2020-03-19 12:13:38,0.0,0.0,0.0,2020-03-20,24.2500,-76.0000,NaN,nan,NaN,NaN,NaN,NaN
7551,nan,"Bahamas, The",2020-03-21 02:43:03,4.0,0.0,0.0,2020-03-21,25.0343,-77.3963,NaN,nan,NaN,NaN,NaN,NaN
7615,nan,The Bahamas,2020-03-19 12:13:38,0.0,0.0,0.0,2020-03-21,24.2500,-76.0000,NaN,nan,NaN,NaN,NaN,NaN


In [241]:
df = df[df["Country/Region"] != 'The Bahamas']

# the mainland china rows shown below are bogus duplicates, so they will be dropped
df[(df["Province/State"].isin(["Gansu", "Hebei"]) ) & ((df.Date == pd.datetime(2020, 3, 11)) | (df.Date == pd.datetime(2020, 3, 12)))]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
4748,Hebei,China,2020-03-11 02:18:14,318.0,6.0,307.0,2020-03-11,39.5490,116.1306,NaN,nan,NaN,NaN,NaN,NaN
4765,Gansu,China,2020-03-11 14:33:03,127.0,2.0,88.0,2020-03-11,37.8099,101.0583,NaN,nan,NaN,NaN,NaN,NaN
4925,Gansu,Mainland China,2020-03-11 02:18:28,0.0,0.0,0.0,2020-03-11,36.0611,103.8343,NaN,nan,NaN,NaN,NaN,NaN
4926,Hebei,Mainland China,2020-03-11 02:18:29,0.0,0.0,0.0,2020-03-11,38.0428,114.5149,NaN,nan,NaN,NaN,NaN,NaN
4965,Hebei,China,2020-03-12 05:33:02,318.0,6.0,308.0,2020-03-12,39.5490,116.1306,NaN,nan,NaN,NaN,NaN,NaN
4983,Gansu,China,2020-03-11 14:33:03,127.0,2.0,88.0,2020-03-12,37.8099,101.0583,NaN,nan,NaN,NaN,NaN,NaN
5146,Gansu,Mainland China,2020-03-11 02:18:28,0.0,0.0,0.0,2020-03-12,36.0611,103.8343,NaN,nan,NaN,NaN,NaN,NaN
5147,Hebei,Mainland China,2020-03-11 02:18:29,0.0,0.0,0.0,2020-03-12,38.0428,114.5149,NaN,nan,NaN,NaN,NaN,NaN


In [242]:
df = df[~((df["Province/State"].isin(["Gansu", "Hebei"])) & \
           (df["Country/Region"] == "Mainland China") & \
           (df.Date.isin([pd.datetime(2020, 3, 11), pd.datetime(2020, 3, 12)])))]

# The country "The Gambia" is a bogus duplicate of "Gambia, The", so it will be deleted
df[df["Country/Region"].isin(["Gambia", "Gambia, The", "The Gambia"]) & \
   (df.Date.isin([pd.datetime(2020, 3, 18), pd.datetime(2020, 3, 19), \
                  pd.datetime(2020, 3, 20), pd.datetime(2020, 3, 21)]))]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
6695,nan,"Gambia, The",2020-03-18 14:13:56,1.0,0.0,0.0,2020-03-18,13.4432,-15.3101,NaN,nan,NaN,NaN,NaN,NaN
6721,nan,The Gambia,2020-03-18 14:13:56,0.0,0.0,0.0,2020-03-18,13.4667,-16.6000,NaN,nan,NaN,NaN,NaN,NaN
6990,nan,"Gambia, The",2020-03-18 14:13:56,1.0,0.0,0.0,2020-03-19,13.4432,-15.3101,NaN,nan,NaN,NaN,NaN,NaN
7013,nan,The Gambia,2020-03-18 14:13:56,0.0,0.0,0.0,2020-03-19,13.4667,-16.6000,NaN,nan,NaN,NaN,NaN,NaN
7286,nan,"Gambia, The",2020-03-18 14:13:56,1.0,0.0,0.0,2020-03-20,13.4432,-15.3101,NaN,nan,NaN,NaN,NaN,NaN
7312,nan,The Gambia,2020-03-18 14:13:56,0.0,0.0,0.0,2020-03-20,13.4667,-16.6000,NaN,nan,NaN,NaN,NaN,NaN
7592,nan,"Gambia, The",2020-03-18 14:13:56,1.0,0.0,0.0,2020-03-21,13.4432,-15.3101,NaN,nan,NaN,NaN,NaN,NaN
7616,nan,The Gambia,2020-03-18 14:13:56,0.0,0.0,0.0,2020-03-21,13.4667,-16.6000,NaN,nan,NaN,NaN,NaN,NaN


In [243]:
df = df[df["Country/Region"] != "The Gambia"]

# the single row with the country "Republic of Ireland" is a bogus duplicate, so it will be deleted
df[df["Country/Region"].isin(["Ireland", "Republic of Ireland"]) & \
   (df.Date.isin([pd.datetime(2020, 3, 7), pd.datetime(2020, 3, 8), pd.datetime(2020, 3, 9)]))]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
3839,nan,Ireland,2020-03-06 20:43:03,18.0,0.0,0.0,2020-03-07,53.1424,-7.6921,NaN,nan,NaN,NaN,NaN,NaN
4066,nan,Republic of Ireland,2020-03-08 21:03:03,21.0,0.0,0.0,2020-03-08,53.1424,-7.6921,NaN,nan,NaN,NaN,NaN,NaN
4070,nan,Ireland,2020-03-08 21:03:03,19.0,0.0,0.0,2020-03-08,53.4167,-8.0000,NaN,nan,NaN,NaN,NaN,NaN
4322,nan,Ireland,2020-03-09 09:53:06,21.0,0.0,0.0,2020-03-09,53.1424,-7.6921,NaN,nan,NaN,NaN,NaN,NaN


In [244]:
df = df[df["Country/Region"] != "Republic of Ireland"]

# The District of Columbia has a duplicate on 3/22/2020, delete the row with Confirmed == 0
df[(df["Province/State"].str.contains("District of Columbia")) & (df.Date == pd.datetime(2020, 3, 22))]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
8954,District of Columbia,US,2020-03-22 23:45:00,102.0,0.0,0.0,2020-03-22,38.904178,-77.01656,11001.0,District of Columbia,0.0,"District of Columbia, District of Columbia, US",NaN,NaN
11032,District of Columbia,US,2020-03-22 23:45:00,102.0,2.0,0.0,2020-03-22,38.904178,-77.01656,11001.0,District of Columbia,0.0,"District of Columbia,District of Columbia,US",NaN,NaN


In [245]:
df = df[~((df["Province/State"].str.contains("District of Columbia")) & \
        (df.Date == pd.datetime(2020, 3, 22)) & \
        (df.Confirmed == 0))]

# check if country names have evolved in combined dataset
sorted(df["Country/Region"].unique().tolist())

[' Azerbaijan',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahamas, The',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cruise Ship',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czech Republic',
 'Czechia',
 'Denmark',
 'Diamond Princess',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fa

In [246]:
# The following alternate names were found by visual inspection of the list
df.replace('Mainland China', 'China', inplace=True)
df.replace('Bahamas, The', 'The Bahamas', inplace=True)
df.replace('Bahamas', 'The Bahamas', inplace=True)
df.replace('Gambia, The', 'Gambia', inplace=True)
df.replace('Hong Kong', 'Hong Kong S.A.R.', inplace=True)
df.replace('Hong Kong SAR', 'Hong Kong S.A.R.', inplace=True)
df.replace('Iran (Islamic Republic of)', 'Iran', inplace=True)
df.replace('Republic of Ireland', 'Ireland', inplace=True)
df.replace('Republic of Korea', 'South Korea', inplace=True)
df.replace('Korea, South', 'South Korea', inplace=True)
df.replace('Republic of Moldova', 'Moldova', inplace=True)
df.replace('Russian Federation', 'Russia', inplace=True)
df.replace('St. Martin', 'Saint Martin', inplace=True)
df.replace('Taiwan*', 'Taiwan', inplace=True)
df.replace('UK', 'United Kingdom', inplace=True)
df.replace('Viet Nam', 'Vietnam', inplace=True)
df.replace("Cote d'Ivoire", 'Ivory Coast', inplace=True)
df.replace(' Azerbaijan', 'Azerbaijan', inplace=True)
df.replace('US', 'United States of America', inplace=True)

# Look for evolving state/province names
for country in sorted(df["Country/Region"].unique().tolist()):
  print(country)

  for state in sorted(df.loc[df["Country/Region"] == country, "Province/State"].unique().tolist()):
    if state != "nan":
      print("|-", state)

    for county in sorted(df.loc[(df["Country/Region"] == country) & (df["Province/State"] == state), "County"].unique().tolist()):
      if county != "nan":
        print("  |-", county)

        #for city in sorted(df.loc[(df["Country/Region"] == country) & \
        #                          (df["Province/State"] == state) & \
        #                          df["County"] == county, "City"].unique().tolist()):
        #  if city != "nan":
        #    print("    |- City:", city)

ter
  |- Wyoming
  |- Yates
|- New York City, NY
|- New York County, NY
|- Norfolk County, MA
|- North Carolina
  |- Alamance
  |- Alexander
  |- Alleghany
  |- Anson
  |- Ashe
  |- Avery
  |- Beaufort
  |- Bertie
  |- Bladen
  |- Brunswick
  |- Buncombe
  |- Burke
  |- Cabarrus
  |- Caldwell
  |- Camden
  |- Carteret
  |- Caswell
  |- Catawba
  |- Chatham
  |- Cherokee
  |- Chowan
  |- Clay
  |- Cleveland
  |- Columbus
  |- Craven
  |- Cumberland
  |- Currituck
  |- Dare
  |- Davidson
  |- Davie
  |- Duplin
  |- Durham
  |- Edgecombe
  |- Forsyth
  |- Franklin
  |- Gaston
  |- Gates
  |- Graham
  |- Granville
  |- Greene
  |- Guilford
  |- Halifax
  |- Harnett
  |- Haywood
  |- Henderson
  |- Hertford
  |- Hoke
  |- Hyde
  |- Iredell
  |- Jackson
  |- Johnston
  |- Jones
  |- Lee
  |- Lenoir
  |- Lincoln
  |- Macon
  |- Madison
  |- Martin
  |- McDowell
  |- Mecklenburg
  |- Mitchell
  |- Montgomery
  |- Moore
  |- Nash
  |- New Hanover
  |- Northampton
  |- Onslow
  |- Orange
  |- Pa

In [247]:
# clean the province/state column after visually inspecting output above
df["Province/State"].replace('None', np.nan, inplace=True)
df[df["Province/State"] == "Recovered"].head(10)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
17676,Recovered,Canada,2020-03-24 23:41:36,0.0,0.0,110.0,2020-03-24,0.0,0.0,NaN,nan,0.0,"Recovered, Canada",NaN,NaN
17677,Recovered,United States of America,2020-03-24 23:37:31,0.0,0.0,348.0,2020-03-24,0.0,0.0,NaN,nan,0.0,"Recovered, US",NaN,NaN
21092,Recovered,Canada,2020-03-25 23:37:36,0.0,0.0,183.0,2020-03-25,0.0,0.0,NaN,nan,0.0,"Recovered, Canada",NaN,NaN
21093,Recovered,United States of America,2020-03-25 23:33:19,0.0,0.0,361.0,2020-03-25,0.0,0.0,NaN,nan,0.0,"Recovered, US",NaN,NaN
24510,Recovered,Canada,2020-03-26 23:53:11,0.0,0.0,184.0,2020-03-26,0.0,0.0,NaN,nan,0.0,"Recovered, Canada",NaN,NaN
24511,Recovered,United States of America,2020-03-26 23:48:35,0.0,0.0,681.0,2020-03-26,0.0,0.0,NaN,nan,0.0,"Recovered, US",NaN,NaN
27938,Recovered,Canada,2020-03-27 23:27:32,0.0,0.0,256.0,2020-03-27,0.0,0.0,NaN,nan,0.0,"Recovered, Canada",NaN,NaN
27939,Recovered,United States of America,2020-03-27 22:14:55,0.0,0.0,869.0,2020-03-27,0.0,0.0,NaN,nan,0.0,"Recovered, US",NaN,NaN
31366,Recovered,Canada,2020-03-28 23:10:00,0.0,0.0,466.0,2020-03-28,0.0,0.0,NaN,nan,0.0,"Recovered, Canada",NaN,NaN
31367,Recovered,United States of America,2020-03-28 23:05:00,0.0,0.0,1072.0,2020-03-28,0.0,0.0,NaN,nan,0.0,"Recovered, US",NaN,NaN


In [248]:
df.loc[(df["Country/Region"] == "Canada") & (df.Date == pd.datetime(2020, 3, 24)), :].Recovered.agg("sum")
# the following result demonstrates that the rows where column 
# Province/State == "Recovered" is an aggregate which can be derived
# there these rows are duplicates and will be deleted

110.0

In [249]:
df.loc[(df["Country/Region"] == "Others") & (df["Province/State"] == "Diamond Princess cruise ship"), "Province/State"] = "Diamond Princess"
df.loc[df["Country/Region"] == "Others", "Country/Region"] = "Cruise Ship"

#Combine US states: Grand Princess and Grand Princess cruise ship
df[df["Province/State"] == "Grand Princess Cruise Ship"].head(10)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
3836,Grand Princess Cruise Ship,United States of America,2020-03-07 01:33:02,21.0,0.0,0.0,2020-03-07,37.6489,-122.6655,NaN,nan,NaN,NaN,NaN,NaN
4067,Grand Princess Cruise Ship,United States of America,2020-03-07 01:33:02,21.0,0.0,0.0,2020-03-08,37.6489,-122.6655,NaN,nan,NaN,NaN,NaN,NaN
4323,Grand Princess Cruise Ship,United States of America,2020-03-07 01:33:02,21.0,0.0,0.0,2020-03-09,37.6489,-122.6655,NaN,nan,NaN,NaN,NaN,NaN


In [250]:
df = df[df["Province/State"] != "Recovered"]

#Diamond Princess country move to Cruise ship country
df[df["Country/Region"] == "Diamond Princess"].head(10)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
21155,nan,Diamond Princess,2020-03-25 23:33:04,712.0,10.0,587.0,2020-03-25,0.0,0.0,NaN,nan,115.0,Diamond Princess,NaN,NaN
24574,nan,Diamond Princess,2020-03-26 23:48:18,712.0,10.0,597.0,2020-03-26,0.0,0.0,NaN,nan,105.0,Diamond Princess,NaN,NaN
28003,nan,Diamond Princess,2020-03-27 23:23:03,712.0,10.0,597.0,2020-03-27,0.0,0.0,NaN,nan,105.0,Diamond Princess,NaN,NaN
31432,nan,Diamond Princess,2020-03-28 23:05:00,712.0,10.0,597.0,2020-03-28,0.0,0.0,NaN,nan,105.0,Diamond Princess,NaN,NaN
34866,nan,Diamond Princess,2020-03-29 23:08:00,712.0,10.0,603.0,2020-03-29,0.0,0.0,NaN,nan,99.0,Diamond Princess,NaN,NaN
38305,nan,Diamond Princess,2020-03-30 22:52:00,712.0,10.0,603.0,2020-03-30,0.0,0.0,NaN,nan,99.0,Diamond Princess,NaN,NaN
40738,nan,Diamond Princess,2020-03-31 23:43:43,712.0,10.0,603.0,2020-03-31,0.0,0.0,NaN,nan,99.0,Diamond Princess,NaN,NaN
43221,nan,Diamond Princess,2020-04-01 21:58:34,712.0,11.0,603.0,2020-04-01,0.0,0.0,NaN,nan,98.0,Diamond Princess,NaN,NaN
45788,nan,Diamond Princess,2020-04-02 23:25:00,712.0,11.0,619.0,2020-04-02,NaN,NaN,NaN,nan,82.0,Diamond Princess,NaN,NaN
48412,nan,Diamond Princess,2020-04-03 22:46:20,712.0,11.0,619.0,2020-04-03,NaN,NaN,NaN,nan,82.0,Diamond Princess,NaN,NaN


In [251]:
df.loc[df["Country/Region"] == "Diamond Princess", "Province/State"] = "Diamond Princess"
df.loc[df["Country/Region"] == "Diamond Princess", "Country/Region"] = "Cruise Ship"
df.loc[df["Province/State"] == "Diamond Princess", "Country/Region"] = "Cruise Ship"

#Others country move to Cruise ship country
df.loc[df["Country/Region"] == "Others", "Province/State"].unique()

array([], dtype=object)

In [252]:
# the Grand Princess cruise ship is listed under the US & Canada
df.loc[df["Province/State"] == "Grand Princess", "Country/Region"].unique()

array(['United States of America', 'Canada'], dtype=object)

In [253]:
# combine these rows and move them under the cruise ship country
for date in df.loc[df["Province/State"] == "Grand Princess", "Date"].unique():
  confirmed_on_date = df.loc[(df["Province/State"] == "Grand Princess") & (df.Date == date), "Confirmed"].agg("sum")
  
  # delete Canadian row
  df = df[~((df["Country/Region"] == "Canada") & (df["Province/State"] == "Grand Princess") & (df.Date == date))]

  # re-assign the remaining US row under the Cruise Ship country
  df.loc[(df["Country/Region"] == "United States of America") & \
         (df["Province/State"] == "Grand Princess") & \
         (df.Date == date), "Confirmed"] = confirmed_on_date
  df.loc[(df["Country/Region"] == "United States of America") & \
         (df["Province/State"] == "Grand Princess") & \
         (df.Date == date), "Country/Region"] = "Cruise Ship"

# three rows have the Grand Princess ship mislabelled
df[df["Province/State"] == "Grand Princess Cruise Ship"]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
3836,Grand Princess Cruise Ship,United States of America,2020-03-07 01:33:02,21.0,0.0,0.0,2020-03-07,37.6489,-122.6655,NaN,nan,NaN,NaN,NaN,NaN
4067,Grand Princess Cruise Ship,United States of America,2020-03-07 01:33:02,21.0,0.0,0.0,2020-03-08,37.6489,-122.6655,NaN,nan,NaN,NaN,NaN,NaN
4323,Grand Princess Cruise Ship,United States of America,2020-03-07 01:33:02,21.0,0.0,0.0,2020-03-09,37.6489,-122.6655,NaN,nan,NaN,NaN,NaN,NaN


In [254]:
df.loc[df["Province/State"] == "Grand Princess Cruise Ship", "Province/State"] = "Grand Princess"
df.loc[df["Province/State"] == "Grand Princess", "Country/Region"] = "Cruise Ship"

In [255]:
# Move Hong Kong from China to Hong Kong
df[(df["Province/State"] == "Hong Kong S.A.R.") & (df["Country/Region"] == "China")].head(5)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude,FIPS,County,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
4766,Hong Kong S.A.R.,China,2020-03-11 18:52:03,126.0,3.0,65.0,2020-03-11,22.3,114.2,NaN,nan,NaN,NaN,NaN,NaN
4982,Hong Kong S.A.R.,China,2020-03-12 01:53:02,129.0,3.0,67.0,2020-03-12,22.3,114.2,NaN,nan,NaN,NaN,NaN,NaN
5175,Hong Kong S.A.R.,China,2020-03-13 13:33:03,134.0,4.0,77.0,2020-03-13,22.3,114.2,NaN,nan,NaN,NaN,NaN,NaN
5435,Hong Kong S.A.R.,China,2020-03-14 12:53:06,140.0,4.0,78.0,2020-03-14,22.3,114.2,NaN,nan,NaN,NaN,NaN,NaN
5684,Hong Kong S.A.R.,China,2020-03-15 18:20:18,145.0,4.0,81.0,2020-03-15,22.3,114.2,NaN,nan,NaN,NaN,NaN,NaN


In [256]:
df.loc[(df["Province/State"] == "Hong Kong S.A.R.") & \
       (df["Country/Region"] == "China"), "Country/Region"] = "Hong Kong S.A.R."
df.loc[(df["Country/Region"] == "Hong Kong S.A.R."), "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Taipei and environs", "Country/Region"] = "Taiwan"
df.loc[df["Country/Region"] == "Taiwan", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Macau SAR", "Country/Region"] = "China"
df.loc[df["Country/Region"] == "Macao SAR", "Country/Region"] = "China"
df.loc[df["Country/Region"] == "Macau", "Country/Region"] = "China"
df.loc[df["Province/State"] == "Bavaria", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Israel", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "France", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Russia", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Germany", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Japan", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Mexico", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "India", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Brazil", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Ukraine", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Colombia", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Peru", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Pakistan", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Italy", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Spain", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Sweden", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Chile", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "United Kingdom", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Denmark", "Province/State"] = "nan"
df.loc[df["Country/Region"] == "Netherlands", "Province/State"] = "nan"
df.loc[(df["Country/Region"] == "Canada") & \
       (df["Province/State"] == "Diamond Princess"), "Province/State"] = "nan"
df.loc[(df["Country/Region"] == "Canada") & \
       (df["Province/State"] == "Nunavut"), "Province/State"] = "nan"
df.loc[(df["Province/State"] == "Wuhan Evacuee"), "Province/State"] = "nan"
df.loc[(df["Province/State"] == "Chicago"), "Province/State"] = "Illinois"
df.loc[(df["Province/State"] == "D.C."), "Province/State"] = "District of Columbia"
df.loc[(df["Province/State"] == "United States Virgin Islands"), "Province/State"] = "Virgin Islands"
df.loc[(df["Country/Region"] == "North Macedonia"), "Country/Region"] = "Macedonia"
df.loc[(df["Country/Region"] == "North Ireland"), "Country/Region"] = "Ireland"
df.loc[(df["Country/Region"] == "Serbia"), "Country/Region"] = "Republic of Serbia"
df.loc[(df["Country/Region"] == "Vatican City"), "Country/Region"] = "Vatican"
df.loc[(df["Country/Region"] == "Holy See"), "Country/Region"] = "Vatican"
df.loc[(df["Country/Region"] == "French Guiana"), "Country/Region"] = "France"
df.loc[(df["Country/Region"] == "Martinique"), "Country/Region"] = "France"
df.loc[(df["Country/Region"] == "Guadeloupe"), "Country/Region"] = "France"
df.loc[(df["Country/Region"] == "Mayotte"), "Country/Region"] = "France"
df.loc[(df["Country/Region"] == "occupied Palestinian territory"), "Country/Region"] = "Palestine"
df.loc[(df["Country/Region"] == "West Bank and Gaza"), "Country/Region"] = "Palestine"
df.loc[(df["Country/Region"] == "Czechia"), "Country/Region"] = "Czech Republic"
df.loc[(df["Country/Region"] == "Curacao"), "Country/Region"] = "Netherlands"
df.loc[(df["Country/Region"] == "Republic of the Congo"), "Country/Region"] = "Republic of Congo"
df.loc[(df["Country/Region"] == "Tanzania"), "Country/Region"] = "United Republic of Tanzania"
df.loc[(df["Country/Region"] == "Cabo Verde"), "Country/Region"] = "Cape Verde"
df.loc[(df["Country/Region"] == "Timor-Leste"), "Country/Region"] = "East Timor"
df.loc[(df["Country/Region"] == "Guinea-Bissau"), "Country/Region"] = "Guinea Bissau"
df.loc[(df["Country/Region"] == "Burma"), "Country/Region"] = "Myanmar"
df.loc[(df["Country/Region"] == "MS Zaandam"), "Country/Region"] = "Cruise Ship"
df.loc[(df.County == "Unassigned"), "County"] = "nan"
df.loc[(df["County"] == "New York City"), "County"] = "New York"
df.loc[(df["County"] == "Bronx"), "County"] = "nan"
df.loc[(df["County"] == "Queens"), "County"] = "nan"

In [257]:
df.loc[df["Country/Region"] == "Canada", "Province/State"].unique()

array(['Ontario', 'British Columbia', 'Toronto, ON', 'London, ON',
       ' Montreal, QC', 'Calgary, Alberta', 'Edmonton, Alberta',
       'Alberta', 'Quebec', 'New Brunswick', 'Manitoba', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
       'Northwest Territories', 'Yukon'], dtype=object)

In [258]:
# Beginning on Feb 1, 2020 through Mar 9, 2020 the data for the US & Canada were reported using
# <city>, <state> in the Province/State field.  Before and after this date range, data was reported
# using only the state name in this field.  The field values within this data range will be modified
# to only specify the state name.

# determine which rows have "<city>, <state>" format
has_city_mask = df["Province/State"].str.find(", ") > -1

# replace the field values with the word following the comma
df.loc[has_city_mask, "Province/State"] = df.loc[has_city_mask, "Province/State"].apply(lambda x: x.split(", ")[1])

# make a dict to replace state/province abbreviations with their full name
# this dict is from https://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
        'OR ': 'Oregon',
        'NE (From Diamond Princess)': 'Nebraska',
        'CA (From Diamond Princess)': 'California',
        'TX (From Diamond Princess)': 'Texas',
        'Unassigned Location (From Diamond Princess)': 'nan',
        'U.S.': 'nan',
        'United States of America': 'nan',
        'ON': 'Ontario',
        'QC': 'Quebec',
        'D.C.': 'District of Columbia'
}
df["Province/State"].replace(states, inplace=True)

# whenever more than one city in the same state/province was recorded on the same date,
# the state will now appear more than once.  These rows need to be aggregated.
df["Country/Region"] = df["Country/Region"].astype(str)
df["Province/State"] = df["Province/State"].astype(str)
df["County"] = df["County"].astype(str)
df2 = df

df2 = df2.groupby(["Country/Region", "Province/State", "County", "Date"]).sum().reset_index()
df2.head(20)

,Country/Region,Province/State,County,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,Active,Incidence_Rate,Case-Fatality_Ratio
0,Afghanistan,nan,nan,2020-02-24,1.0,0.0,0.0,0.0000,0.00,0.0,0.0,0.0,0.0
1,Afghanistan,nan,nan,2020-02-25,1.0,0.0,0.0,0.0000,0.00,0.0,0.0,0.0,0.0
2,Afghanistan,nan,nan,2020-02-26,1.0,0.0,0.0,0.0000,0.00,0.0,0.0,0.0,0.0
3,Afghanistan,nan,nan,2020-02-27,1.0,0.0,0.0,0.0000,0.00,0.0,0.0,0.0,0.0
4,Afghanistan,nan,nan,2020-02-28,1.0,0.0,0.0,0.0000,0.00,0.0,0.0,0.0,0.0
5,Afghanistan,nan,nan,2020-02-29,1.0,0.0,0.0,0.0000,0.00,0.0,0.0,0.0,0.0
6,Afghanistan,nan,nan,2020-03-01,1.0,0.0,0.0,33.0000,65.00,0.0,0.0,0.0,0.0
7,Afghanistan,nan,nan,2020-03-02,1.0,0.0,0.0,33.0000,65.00,0.0,0.0,0.0,0.0
8,Afghanistan,nan,nan,2020-03-03,1.0,0.0,0.0,33.0000,65.00,0.0,0.0,0.0,0.0
9,Afghanistan,nan,nan,2020-03-04,1.0,0.0,0.0,33.0000,65.00,0.0,0.0,0.0,0.0


In [259]:
t1 = time.time()
(t1 - t0)/60

5.664904189109802

In [260]:
np.sort(df2["Country/Region"].unique().tolist())

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Channel Islands', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo (Brazzaville)',
       'Congo (Kinshasa)', 'Costa Rica', 'Croatia', 'Cruise Ship', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland',
       'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana

In [261]:
# calculate new cases for each region on each date
df = df2
df["Country/Region"] = df["Country/Region"].astype(str)
df["Province/State"] = df["Province/State"].astype(str)
df["County"] = df["County"].astype(str)
all_csc_per_date_df = pd.DataFrame(columns=["Country/Region", "Province/State", "County"])
for country in sorted(df["Country/Region"].unique().tolist()):
  for state in sorted(df.loc[df["Country/Region"] == country, "Province/State"].unique().tolist()):
    for county in sorted(df.loc[(df["Country/Region"] == country) & \
                                (df["Province/State"] == state), "County"].unique().tolist()):
      
      # isolate corresponding country-state-county combination
      csc_df = df.loc[(df["Country/Region"] == country) & \
                           (df["Province/State"] == state) & \
                           (df["County"] == county), ["Date", "Confirmed", "Recovered", "Deaths"]]
      csc_df.Date = csc_df.Date.dt.strftime("%Y-%m-%d")

      # pivot into time series array (each column is a date)
      csc_conf_arr = csc_df.drop(labels=["Recovered", "Deaths"], axis=1).values.transpose()
      csc_recv_arr = csc_df.drop(labels=["Confirmed", "Deaths"], axis=1).values.transpose()
      csc_death_arr = csc_df.drop(labels=["Confirmed", "Recovered"], axis=1).values.transpose()

      # calcuate new confirmations for each date
      all_arrs = np.zeros([3, csc_conf_arr.shape[0],  csc_conf_arr.shape[1]])
      for i, arr in enumerate([csc_conf_arr, csc_recv_arr, csc_death_arr]):
        yesterday = arr[:, 0:-1]
        today = arr[:, 1:]
        arr_per_date = arr[0,:]
        arr_per_date = np.vstack([arr_per_date, np.hstack([np.nan, today[1,:] - yesterday[1,:]])])

        # return array back to dataframe
        arr_per_date_df = pd.DataFrame(columns=arr_per_date[0,:].astype(str))
        arr_per_date_df = arr_per_date_df.append(pd.Series(dict(zip(arr_per_date_df.columns, arr_per_date[1,:]))), ignore_index=True)
        arr_per_date_df["Country/Region"] = country
        arr_per_date_df["Province/State"] = state
        arr_per_date_df["County"] = county

        if i == 0:
          arr_per_date_df["Var"] = "ConfirmedPerDate"
        if i == 1:
          arr_per_date_df["Var"] = "RecoveredPerDate"
        if i == 2:
          arr_per_date_df["Var"] = "DeathsPerDate"

        # add this combo to collection
        #print(arr_per_date_df.head(1))
        #print("---------------------------")
        all_csc_per_date_df = all_csc_per_date_df.append(arr_per_date_df, ignore_index=True)

    # Beginning on Mar 22 the US had data broken down at the county level.
    # This will cause a problem on the 22 & 23 because there is no prior county
    # data to reference, causing the standard calculation to fail.  Each state will
    # be corrected here.
    if (country == "United States of America"):
        
      # get the df row for the entire state to get total state cases on Mar 21
      state_21_df = df.loc[(df["Country/Region"] == country) & (df["Province/State"] == state) & \
                           (df.Date == pd.datetime(2020, 3, 21)), ["Confirmed", "Recovered", "Deaths"]]
      #print(state)
      if state_21_df.size > 0:
        state_21_arr = state_21_df.values[0,:]
      else:
        state_21_arr =np.array([0, 0, 0])
      #print(state_21_arr)
      # get the df rows for all the state's counties to get total cases on Mar 22
      counties_df = df.loc[(df["Country/Region"] == country) & (df["Province/State"] == state) & \
                           (df.Date == pd.datetime(2020, 3, 22)), ["Confirmed", "Recovered", "Deaths"]]
      counties_22_arr = np.array([counties_df["Confirmed"].sum(), counties_df["Recovered"].sum(),
                                     counties_df["Deaths"].sum()])
      #print(counties_22_arr)
      # calculate the increase in cases ocurring on Mar 22
      state_per_date_22_arr = counties_22_arr - state_21_arr
      #print(state_per_date_22_arr)
      
      # correct the value on Mar 22
      all_csc_per_date_df.loc[(all_csc_per_date_df["Country/Region"] == "United States of America") & \
                              (all_csc_per_date_df["Province/State"] == state) & \
                              (all_csc_per_date_df.County == "nan") & \
                              (all_csc_per_date_df.Var == "ConfirmedPerDate"), "2020-03-22"] = state_per_date_22_arr[0]
      all_csc_per_date_df.loc[(all_csc_per_date_df["Country/Region"] == "United States of America") & \
                              (all_csc_per_date_df["Province/State"] == state) & \
                              (all_csc_per_date_df.County == "nan") & \
                              (all_csc_per_date_df.Var == "RecoveredPerDate"), "2020-03-22"] = state_per_date_22_arr[1]
      all_csc_per_date_df.loc[(all_csc_per_date_df["Country/Region"] == "United States of America") & \
                              (all_csc_per_date_df["Province/State"] == state) & \
                              (all_csc_per_date_df.County == "nan") & \
                              (all_csc_per_date_df.Var == "DeathsPerDate"), "2020-03-22"] = state_per_date_22_arr[2]


In [262]:
all_csc_per_date_df.loc[(all_csc_per_date_df["Province/State"] == "Ohio") & (all_csc_per_date_df.County == "nan"), ["2020-03-21", "2020-03-22", "2020-03-23", "2020-03-24"]].head(3)

,2020-03-21,2020-03-22,2020-03-23,2020-03-24
7302,75.0,108,NaN,NaN
7303,0.0,0,NaN,NaN
7304,1.0,0,NaN,NaN


In [263]:
print(arr_per_date_df.columns)
arr_per_date_df

Index(['2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23', '2020-03-24',
       '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29',
       ...
       '2020-08-04', '2020-08-05', '2020-08-06', '2020-08-07', '2020-08-08',
       '2020-08-09', 'Country/Region', 'Province/State', 'County', 'Var'],
      dtype='object', length=147)


,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,...,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,Country/Region,Province/State,County,Var
0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,18.0,0.0,2.0,Zimbabwe,nan,nan,DeathsPerDate


In [264]:
all_csc_per_date_df.loc[(all_csc_per_date_df["Country/Region"] == "France") & \
                    
                    (all_csc_per_date_df.Var == "ConfirmedPerDate"), ["Province/State", "2020-03-24"]].values
                    #(all_csc_per_date_df.County == "nan")]

array([['nan', 2499.0]], dtype=object)

In [265]:
all_csc_per_date_df["Country/Region"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Channel Islands', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo (Brazzaville)',
       'Congo (Kinshasa)', 'Costa Rica', 'Croatia', 'Cruise Ship', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland',
       'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana

In [266]:
all_csc_conf_per_date_df = all_csc_per_date_df[all_csc_per_date_df.Var == "ConfirmedPerDate"].\
                           drop("Var", axis=1).melt(id_vars=["Country/Region", "Province/State", "County"],
                                                    var_name="Date", value_name="ConfirmedPerDate")
all_csc_conf_per_date_df.Date = pd.to_datetime(all_csc_conf_per_date_df.Date)

all_csc_recv_per_date_df = all_csc_per_date_df[all_csc_per_date_df.Var == "RecoveredPerDate"].\
                           drop("Var", axis=1).melt(id_vars=["Country/Region", "Province/State", "County"],
                                                    var_name="Date", value_name="RecoveredPerDate")
all_csc_recv_per_date_df.Date = pd.to_datetime(all_csc_recv_per_date_df.Date)

all_csc_death_per_date_df = all_csc_per_date_df[all_csc_per_date_df.Var == "DeathsPerDate"].\
                            drop("Var", axis=1).melt(id_vars=["Country/Region", "Province/State", "County"],
                                                     var_name="Date", value_name="DeathsPerDate")
all_csc_death_per_date_df.Date = pd.to_datetime(all_csc_death_per_date_df.Date)
all_csc_conf_per_date_df["Country/Region"].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Channel Islands', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo (Brazzaville)',
       'Congo (Kinshasa)', 'Costa Rica', 'Croatia', 'Cruise Ship', 'Cuba',
       'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland',
       'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana

In [267]:
# outer join the confirmed-per-date data with the clean dataframe
df = df.merge(all_csc_conf_per_date_df, how="outer", on=["Country/Region", "Province/State", "County", "Date"])
df = df.merge(all_csc_recv_per_date_df, how="outer", on=["Country/Region", "Province/State", "County", "Date"])
df = df.merge(all_csc_death_per_date_df, how="outer", on=["Country/Region", "Province/State", "County", "Date"])

In [268]:
# Beginning on March 22 the US county data for Confirmed, Deaths & Recovered 
# need to be aggregated for state-wide values.  ConfirmedPerDate, DeathsPerDate &
# RecoveredPerDate need to be aggregated beginning on March 23.
us_cnty_mask = (df["Country/Region"] == "United States of America") & (df.County != "nan")
us_state_mask = (df["Country/Region"] == "United States of America") & (df.County == "nan")
cumul_vars = ["Confirmed", "Recovered", "Deaths"]
perday_vars = ["ConfirmedPerDate", "RecoveredPerDate", "DeathsPerDate"]
for state in np.sort(df.loc[us_cnty_mask, "Province/State"].unique()):
    for date in np.sort(df.loc[us_cnty_mask & (df["Province/State"] == state), "Date"].unique()):
        state_day_mask = us_state_mask & (df["Province/State"] == state) & (df.Date == date)
        counties_day_mask = us_cnty_mask & (df["Province/State"] == state) & (df.Date == date)
        if pd.to_datetime(date) > pd.datetime(2020, 3, 21):
            df.loc[state_day_mask, cumul_vars] = df.loc[counties_day_mask, cumul_vars].sum().values
        
        if pd.to_datetime(date) > pd.datetime(2020, 3, 22):
            df.loc[state_day_mask, perday_vars] = df.loc[counties_day_mask, perday_vars].sum().values

In [269]:
# make a column for US state abbreviations (used by Dash)
states_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
        'nan': 'nan'
}

inverted_state_dict = {value: key for key, value in states_dict.items()}
df["USstateAbbr"] = df["Province/State"].replace(inverted_state_dict)
df.loc[df["Country/Region"] != "United States of America", "USstateAbbr"] = "nan"
df.head(3)

,Country/Region,Province/State,County,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,Active,Incidence_Rate,Case-Fatality_Ratio,ConfirmedPerDate,RecoveredPerDate,DeathsPerDate,USstateAbbr
0,Afghanistan,nan,nan,2020-02-24,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,nan
1,Afghanistan,nan,nan,2020-02-25,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,nan
2,Afghanistan,nan,nan,2020-02-26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,nan


In [270]:
df3 = df
#df = df3

In [271]:
# calculate US county per capita variables
us_pop_df = pd.read_pickle(r"C:\Users\adiad\Anaconda3\envs\CovidApp36\covidapp\data_clean\us_pop_df.pkl")
us_pop_df.head(3)

,State,County,FIPS,Population
0,Alabama,Alabama,NaN,4903185
1,Alabama,Autauga County,01001,55869
2,Alabama,Baldwin County,01003,223234


In [272]:
# create a dict to trnsform us counties to their population
us_cnty_pop_dict = {}
us_cnty_pop_df = us_pop_df[us_pop_df.FIPS.notna()]
for fips in us_cnty_pop_df.FIPS:
    pop = us_cnty_pop_df.loc[us_cnty_pop_df.FIPS == fips, "Population"].values[0]
    us_cnty_pop_dict.update({fips:pop})

print("FIPS  Population")
for x in list(us_cnty_pop_dict)[:3]:
    print(x, us_cnty_pop_dict[x])

FIPS  Population
01001 55869
01003 223234
01005 24686


In [273]:
# FIPS is supposed to be 5 numeric characters
# with 0 padding on the left
df.FIPS[df.FIPS.isna() | df.FIPS.isnull()] = 0
df.FIPS = df.FIPS.astype(int).astype(str)
df.loc[(df.FIPS.str.len() > 1) & (df.FIPS.str.len() < 5), "FIPS"] = \
    df.loc[(df.FIPS.str.len() > 1) & \
           (df.FIPS.str.len() < 5), "FIPS"].str.pad(width=5, side="left", fillchar="0")
df.FIPS[df.FIPS.str.startswith("8") | df.FIPS.str.startswith("9")] = "0"
df.loc[df.FIPS == "22002", "FIPS"] = "11001"

In [274]:
# calculate per capita values for US counties
case_vars = ["Confirmed", "Recovered", "Deaths"]
cap_vars = [var + "PerCapita" for var in case_vars]
date_vars = [var + "PerDate" for var in case_vars]
cap_date_vars = [var + "PerCapita" for var in date_vars]
per_day_cap_factor = 1

# create per capita columns with arbitrary float values
for var in cap_vars + cap_date_vars:
    df[var] = np.nan

# add population data to dataframe
df["Population"] = df.FIPS
df.Population = df.Population.replace(us_cnty_pop_dict)

# calculate per capita values
us_cnty_mask = (df["Country/Region"] == "United States of America") & \
               (df.Date > pd.datetime(2020, 3, 21)) & (df.FIPS.isin(us_cnty_pop_dict.keys()))
pop_arr = df.loc[us_cnty_mask, "Population"].values
df.loc[us_cnty_mask, cap_vars] = df.loc[us_cnty_mask, case_vars].values / \
                                 np.stack((pop_arr for i in range(3)), axis=1)
df.loc[us_cnty_mask, cap_date_vars] = df.loc[us_cnty_mask, date_vars].values * per_day_cap_factor / \
                                      np.stack((pop_arr for i in range(3)), axis=1)

df[us_cnty_mask].drop(["Latitude", "Longitude"], axis=1).head(10)

,Country/Region,Province/State,County,Date,Confirmed,Deaths,Recovered,FIPS,Active,Incidence_Rate,...,RecoveredPerDate,DeathsPerDate,USstateAbbr,ConfirmedPerCapita,RecoveredPerCapita,DeathsPerCapita,ConfirmedPerDatePerCapita,RecoveredPerDatePerCapita,DeathsPerDatePerCapita,Population
37791,United States of America,Alabama,Autauga,2020-03-22,0.0,0.0,0.0,01001,0.0,0.0,...,NaN,NaN,AL,0.000000,0.0,0.0,NaN,NaN,NaN,55869
37792,United States of America,Alabama,Autauga,2020-03-23,0.0,0.0,0.0,01001,0.0,0.0,...,0,0,AL,0.000000,0.0,0.0,0.000000,0.0,0.0,55869
37793,United States of America,Alabama,Autauga,2020-03-24,1.0,0.0,0.0,01001,0.0,0.0,...,0,0,AL,0.000018,0.0,0.0,0.000018,0.0,0.0,55869
37794,United States of America,Alabama,Autauga,2020-03-25,4.0,0.0,0.0,01001,0.0,0.0,...,0,0,AL,0.000072,0.0,0.0,0.000054,0.0,0.0,55869
37795,United States of America,Alabama,Autauga,2020-03-26,6.0,0.0,0.0,01001,0.0,0.0,...,0,0,AL,0.000107,0.0,0.0,0.000036,0.0,0.0,55869
37796,United States of America,Alabama,Autauga,2020-03-27,6.0,0.0,0.0,01001,0.0,0.0,...,0,0,AL,0.000107,0.0,0.0,0.000000,0.0,0.0,55869
37797,United States of America,Alabama,Autauga,2020-03-28,6.0,0.0,0.0,01001,0.0,0.0,...,0,0,AL,0.000107,0.0,0.0,0.000000,0.0,0.0,55869
37798,United States of America,Alabama,Autauga,2020-03-29,6.0,0.0,0.0,01001,0.0,0.0,...,0,0,AL,0.000107,0.0,0.0,0.000000,0.0,0.0,55869
37799,United States of America,Alabama,Autauga,2020-03-30,6.0,0.0,0.0,01001,0.0,0.0,...,0,0,AL,0.000107,0.0,0.0,0.000000,0.0,0.0,55869
37800,United States of America,Alabama,Autauga,2020-03-31,7.0,0.0,0.0,01001,0.0,0.0,...,0,0,AL,0.000125,0.0,0.0,0.000018,0.0,0.0,55869


In [275]:
df.FIPS[(~df.FIPS.isin(us_cnty_pop_dict.keys()))].unique()

array(['0', '00250', '60000', '66000', '00066', '72086', '69000', '00069',
       '72001', '72003', '72005', '72007', '72009', '72011', '72013',
       '72015', '72017', '72019', '72021', '72023', '72025', '72027',
       '72029', '72031', '72033', '72035', '72037', '72039', '72041',
       '72043', '72045', '72047', '72049', '72051', '72053', '72054',
       '72055', '72057', '72059', '72061', '72063', '72065', '72067',
       '72069', '72071', '72073', '72075', '72077', '72079', '72081',
       '72083', '72085', '72087', '72089', '72091', '72093', '72095',
       '72097', '72099', '72101', '72103', '72105', '72107', '72888',
       '72109', '72111', '72113', '72115', '72117', '72119', '72121',
       '72123', '72125', '72127', '72129', '72131', '72133', '72135',
       '72137', '72139', '72141', '72143', '72145', '72147', '72149',
       '72151', '72153', '00072', '72999', '78000', '00078'], dtype=object)

In [276]:
df4 = df
#df = df4
df4[(df4["Province/State"] == "Ohio") & (df4.County == "nan")].sort_values(by=["Date"])

,Country/Region,Province/State,County,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,...,RecoveredPerDate,DeathsPerDate,USstateAbbr,ConfirmedPerCapita,RecoveredPerCapita,DeathsPerCapita,ConfirmedPerDatePerCapita,RecoveredPerDatePerCapita,DeathsPerDatePerCapita,Population
712707,United States of America,Ohio,nan,2020-01-22,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,OH,NaN,NaN,NaN,NaN,NaN,NaN,0
702130,United States of America,Ohio,nan,2020-01-23,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,OH,NaN,NaN,NaN,NaN,NaN,NaN,0
716236,United States of America,Ohio,nan,2020-01-24,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,OH,NaN,NaN,NaN,NaN,NaN,NaN,0
705655,United States of America,Ohio,nan,2020-01-25,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,OH,NaN,NaN,NaN,NaN,NaN,NaN,0
709177,United States of America,Ohio,nan,2020-01-26,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,OH,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325595,United States of America,Ohio,nan,2020-08-05,96305.0,3596.0,0.0,0.0,0.0,0,...,0,26,OH,NaN,NaN,NaN,NaN,NaN,NaN,0
325596,United States of America,Ohio,nan,2020-08-06,97471.0,3618.0,0.0,0.0,0.0,0,...,0,22,OH,NaN,NaN,NaN,NaN,NaN,NaN,0
325597,United States of America,Ohio,nan,2020-08-07,98675.0,3652.0,0.0,0.0,0.0,0,...,0,34,OH,NaN,NaN,NaN,NaN,NaN,NaN,0
325598,United States of America,Ohio,nan,2020-08-08,99969.0,3668.0,0.0,0.0,0.0,0,...,0,16,OH,NaN,NaN,NaN,NaN,NaN,NaN,0


In [277]:
# create a dict to trnsform us states to their population
us_state_mask = (df["Country/Region"] == "United States of America") & (df.County == "nan")

us_state_pop_dict = {}
for state in np.sort(df.loc[us_state_mask, "Province/State"].unique().tolist()):
    fips_ls = np.sort(df.loc[us_cnty_mask & (df["Province/State"] == state), "FIPS"].unique())
    pop = us_cnty_pop_df.loc[us_cnty_pop_df.FIPS.isin(fips_ls), "Population"].values.sum()
    if pop > 0:
        us_state_pop_dict.update({state:pop})
    else:
        us_state_pop_dict.update({state:np.nan})

print("State  Population")
for x in list(us_state_pop_dict)[:3]:
    print(x, us_state_pop_dict[x])

State  Population
Alabama 4903185
Alaska 731545
American Samoa nan


In [278]:
# calculate per capita values for entire US states
us_cnty_mask = (df["Country/Region"] == "United States of America") & \
               (df.Date > pd.datetime(2020, 3, 21)) & (df.County != "nan")

df.loc[us_state_mask, "Population"] = df.loc[us_state_mask, "Province/State"].replace(us_state_pop_dict)
pop_arr = df.loc[us_state_mask, "Population"].values
df.loc[us_state_mask, cap_vars] = df.loc[us_state_mask, case_vars].values / \
                                         np.stack((pop_arr for i in range(3)), axis=1)
df.loc[us_state_mask, cap_date_vars] = df.loc[us_state_mask, date_vars].values * per_day_cap_factor / \
                                              np.stack((pop_arr for i in range(3)), axis=1)

df[us_state_mask & (df.Date == pd.datetime(2020, 4, 20))].head(30)

,Country/Region,Province/State,County,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,...,RecoveredPerDate,DeathsPerDate,USstateAbbr,ConfirmedPerCapita,RecoveredPerCapita,DeathsPerCapita,ConfirmedPerDatePerCapita,RecoveredPerDatePerCapita,DeathsPerDatePerCapita,Population
47276,United States of America,Alabama,nan,2020-04-20,5079.0,163.0,0.0,0.0000,0.0000,0,...,0,6,AL,0.001036,0.0,0.000033,0.000039,0.0,1.223694e-06,4903185.0
49963,United States of America,Alaska,nan,2020-04-20,321.0,5.0,0.0,0.0000,0.0000,0,...,0,0,AK,0.000439,0.0,0.000007,0.000003,0.0,0.000000e+00,731545.0
52282,United States of America,Arizona,nan,2020-04-20,5068.0,190.0,0.0,0.0000,0.0000,0,...,0,7,AZ,0.000696,0.0,0.000026,0.000019,0.0,9.617079e-07,7278717.0
62719,United States of America,Arkansas,nan,2020-04-20,1928.0,41.0,0.0,0.0000,0.0000,0,...,0,2,AR,0.000639,0.0,0.000014,0.000064,0.0,6.627336e-07,3017804.0
70780,United States of America,California,nan,2020-04-20,33686.0,1223.0,0.0,0.0000,0.0000,0,...,0,48,CA,0.000853,0.0,0.000031,0.000057,0.0,1.214814e-06,39512223.0
79391,United States of America,Colorado,nan,2020-04-20,9616.0,419.0,0.0,0.0000,0.0000,0,...,0,0,CO,0.001670,0.0,0.000073,0.000000,0.0,0.000000e+00,5758736.0
80590,United States of America,Connecticut,nan,2020-04-20,19301.0,1327.0,0.0,0.0000,0.0000,0,...,0,204,CT,0.005414,0.0,0.000372,0.000524,0.0,5.721840e-05,3565287.0
81157,United States of America,Delaware,nan,2020-04-20,2714.0,72.0,0.0,0.0000,0.0000,0,...,0,5,DE,0.002787,0.0,0.000074,0.000212,0.0,5.134714e-06,973764.0
90860,United States of America,Florida,nan,2020-04-20,27056.0,822.0,0.0,0.0000,0.0000,0,...,0,48,FL,0.001260,0.0,0.000038,0.000035,0.0,2.234872e-06,21477737.0
113395,United States of America,Georgia,nan,2020-04-20,19111.0,773.0,0.0,0.0000,0.0000,0,...,0,86,GA,0.001800,0.0,0.000073,0.000100,0.0,8.099894e-06,10617423.0


In [279]:
df5 = df
#df = df5

In [280]:
# aggregate US states into whole country rows
all_us_mask = (df["Country/Region"] == "United States of America") & (df.County == "nan")
agg_pop = us_cnty_pop_df.Population.sum()
pop_ls = [agg_pop for i in range(3)]
factor = np.append(np.array([1, 1, 1]), per_day_cap_factor*np.array([1, 1, 1]))
row_ls = []

for date in np.sort(df.loc[all_us_mask, "Date"].unique()):

    # start a new row by duplicating an extant row
    all_us_agg_row = df[all_us_mask & (df.Date == date)].head(1)

    # calculate aggregates and per capita values
    agg_vars = df.loc[all_us_mask & (df.Date == date), case_vars].sum().values
    all_us_agg_row[case_vars] = agg_vars
    all_us_agg_row[cap_vars] = agg_vars / pop_ls

    agg_vars = df.loc[all_us_mask & (df.Date == date), date_vars].sum().values
    all_us_agg_row[date_vars] = agg_vars
    all_us_agg_row[cap_date_vars] = agg_vars * per_day_cap_factor / pop_ls

    # tweak row to reflect that its for th whole country
    all_us_agg_row.County = "nan"
    all_us_agg_row.FIPS = "0"
    all_us_agg_row["Province/State"] = "nan"
    all_us_agg_row.USstateAbbr = "nan"
    #print(all_us_agg_row[case_vars + date_vars + cap_vars + cap_date_vars].values)

    # add row to dataframe
    row_ls.append(all_us_agg_row)

# combine new rows into dataframe
all_us_df = pd.concat(row_ls, ignore_index=True)

# delete extant rows for whole country
df = df[~(all_us_mask & (df["Province/State"] == "nan"))].reset_index(drop=True)

# add each new row to end of dataframe
for row in range(all_us_df.shape[0]):
    df.loc[len(df.index)] = all_us_df.iloc[row,:].values

In [281]:
# calculate Australia & China state/province per capita variables
acc_pop_df = pd.read_pickle(r"C:\Users\adiad\Anaconda3\envs\CovidApp36\covidapp\data_clean\australia_china_canada_pop_df.pkl")
acc_pop_df.head(20)

,State,Population,Country
0,New South Wales,8089526.0,Australia
1,Queensland,5095100.0,Australia
2,South Australia,1751693.0,Australia
3,Tasmania,534281.0,Australia
4,Victoria,6594804.0,Australia
5,Western Australia,2621680.0,Australia
6,Australian Capital Territory,426709.0,Australia
7,Jervis Bay Territory,405.0,Australia
8,Northern Territory,245869.0,Australia
9,NaN,NaN,Australia


In [282]:
acc_pop_df.State.to_list()

['New South Wales',
 'Queensland',
 'South Australia',
 'Tasmania',
 'Victoria',
 'Western Australia',
 'Australian Capital Territory',
 'Jervis Bay Territory',
 'Northern Territory',
 nan,
 'Anhui',
 'Beijing',
 'Chongqing',
 'Fujian',
 'Guangdong',
 'Gansu',
 'Guangxi Zhuang Autonomous Region',
 'Guizhou',
 'Henan',
 'Hubei',
 'Hebei',
 'Hainan',
 'Hong Kong Special Administrative Region',
 'Heilongjiang',
 'Hunan',
 'Jilin',
 'Jiangsu',
 'Jiangxi',
 'Liaoning',
 'Macau Special Administrative Region',
 'Inner Mongolia Autonomous Region',
 'Ningxia Hui Autonomous Region',
 'Qinghai',
 'Sichuan',
 'Shandong',
 'Shanghai',
 'Shaanxi',
 'Shanxi',
 'Tianjin',
 'Taiwan',
 'Xinjiang Uyghur Autonomous Region',
 'Tibet Autonomous Region',
 'Yunnan',
 'Zhejiang',
 'Ontario',
 'Quebec',
 'British Columbia',
 'Alberta',
 'Manitoba',
 'Saskatchewan',
 'Nova Scotia',
 'New Brunswick',
 'Newfoundland and Labrador',
 'Prince Edward Island',
 'Northwest Territories',
 'Nunavut',
 'Yukon']

In [283]:
states = df.loc[df["Country/Region"].isin(["Australia", "China", "Canada"]), \
                "Province/State"].unique().tolist()
states

['Australian Capital Territory',
 'External territories',
 'From Diamond Princess',
 'Jervis Bay Territory',
 'New South Wales',
 'Northern Territory',
 'Queensland',
 'South Australia',
 'Tasmania',
 'Victoria',
 'Western Australia',
 'nan',
 'Alberta',
 'British Columbia',
 'Manitoba',
 'New Brunswick',
 'Newfoundland and Labrador',
 'Northwest Territories',
 'Nova Scotia',
 'Ontario',
 'Prince Edward Island',
 'Quebec',
 'Saskatchewan',
 'Yukon',
 'Anhui',
 'Beijing',
 'Chongqing',
 'Fujian',
 'Gansu',
 'Guangdong',
 'Guangxi',
 'Guizhou',
 'Hainan',
 'Hebei',
 'Heilongjiang',
 'Henan',
 'Hubei',
 'Hunan',
 'Inner Mongolia',
 'Jiangsu',
 'Jiangxi',
 'Jilin',
 'Liaoning',
 'Macau',
 'Ningxia',
 'Qinghai',
 'Shaanxi',
 'Shandong',
 'Shanghai',
 'Shanxi',
 'Sichuan',
 'Tianjin',
 'Tibet',
 'Xinjiang',
 'Yunnan',
 'Zhejiang']

In [284]:
acc_pop_df = acc_pop_df[acc_pop_df.State.notna()]

# print epidemiology states/provinces which aren't matched in the population data
[state for state in states if state not in acc_pop_df.State.to_list()]

['External territories',
 'From Diamond Princess',
 'nan',
 'Guangxi',
 'Inner Mongolia',
 'Macau',
 'Ningxia',
 'Tibet',
 'Xinjiang']

In [285]:
# correct the unmatched states/provinces in the population data
acc_pop_df.loc[acc_pop_df.State == "Guangxi Zhuang Autonomous Region", "State"] = "Guangxi"
acc_pop_df.loc[acc_pop_df.State == "Inner Mongolia Autonomous Region", "State"] = "Inner Mongolia"
acc_pop_df.loc[acc_pop_df.State == "Macau Special Administrative Region", "State"] = "Macau"
acc_pop_df.loc[acc_pop_df.State == "Ningxia Hui Autonomous Region", "State"] = "Ningxia"
acc_pop_df.loc[acc_pop_df.State == "Tibet Autonomous Region", "State"] = "Tibet"
acc_pop_df.loc[acc_pop_df.State == "Xinjiang Uyghur Autonomous Region", "State"] = "Xinjiang"
acc_pop_df.loc[acc_pop_df.State == "Hong Kong Special Administrative Region", "State"] = "Hong Kong S.A.R."

In [286]:
# create a dict to transform states/provinces to their population
acc_pop_dict = {}
for province in acc_pop_df.State.unique():
    print(province)
    print(acc_pop_df[acc_pop_df.State == province])
    print("   ")
    pop = acc_pop_df.loc[acc_pop_df.State == province, "Population"].values[0]
    acc_pop_dict.update({province:pop})

print("State, Population")
for x in list(acc_pop_dict)[:3]:
    print(x, ",", acc_pop_dict[x])

New South Wales
             State  Population    Country
0  New South Wales   8089526.0  Australia
   
Queensland
        State  Population    Country
1  Queensland   5095100.0  Australia
   
South Australia
             State  Population    Country
2  South Australia   1751693.0  Australia
   
Tasmania
      State  Population    Country
3  Tasmania    534281.0  Australia
   
Victoria
      State  Population    Country
4  Victoria   6594804.0  Australia
   
Western Australia
               State  Population    Country
5  Western Australia   2621680.0  Australia
   
Australian Capital Territory
                          State  Population    Country
6  Australian Capital Territory    426709.0  Australia
   
Jervis Bay Territory
                  State  Population    Country
7  Jervis Bay Territory       405.0  Australia
   
Northern Territory
                State  Population    Country
8  Northern Territory    245869.0  Australia
   
Anhui
    State  Population Country
10  Anhui  59500

In [287]:
df6 = df
#df = df6

In [288]:
# calculating per capita values for australian, chinese & canadian states/provinces
acc_mask = df["Country/Region"].isin(["Australia", "China", "Canada"]) & \
           (df["Province/State"].isin(acc_pop_dict.keys()))
df.loc[acc_mask, "Population"] = df.loc[acc_mask, "Province/State"].replace(acc_pop_dict)
pop_arr = df.loc[acc_mask, "Population"].values
df.loc[acc_mask, cap_vars] = df.loc[acc_mask, case_vars].values / \
                             np.stack((pop_arr for i in range(3)), axis=1)
df.loc[acc_mask, cap_date_vars] = df.loc[acc_mask, date_vars].values * per_day_cap_factor / \
                                         np.stack((pop_arr for i in range(3)), axis=1)

df[acc_mask].head(5)

,Country/Region,Province/State,County,Date,Confirmed,Deaths,Recovered,Latitude,Longitude,FIPS,...,RecoveredPerDate,DeathsPerDate,USstateAbbr,ConfirmedPerCapita,RecoveredPerCapita,DeathsPerCapita,ConfirmedPerDatePerCapita,RecoveredPerDatePerCapita,DeathsPerDatePerCapita,Population
1272,Australia,Australian Capital Territory,nan,2020-03-13,1.0,0.0,0.0,-35.4735,149.0124,0,...,NaN,NaN,nan,0.000002,0.0,0.0,NaN,NaN,NaN,426709.0
1273,Australia,Australian Capital Territory,nan,2020-03-14,1.0,0.0,0.0,-35.4735,149.0124,0,...,0,0,nan,0.000002,0.0,0.0,0.000000,0.0,0.0,426709.0
1274,Australia,Australian Capital Territory,nan,2020-03-15,1.0,0.0,0.0,-35.4735,149.0124,0,...,0,0,nan,0.000002,0.0,0.0,0.000000,0.0,0.0,426709.0
1275,Australia,Australian Capital Territory,nan,2020-03-16,2.0,0.0,0.0,-35.4735,149.0124,0,...,0,0,nan,0.000005,0.0,0.0,0.000002,0.0,0.0,426709.0
1276,Australia,Australian Capital Territory,nan,2020-03-17,2.0,0.0,0.0,-35.4735,149.0124,0,...,0,0,nan,0.000005,0.0,0.0,0.000000,0.0,0.0,426709.0


In [289]:
# aggregate Australian/Chinese/Canadian states/provinces into whole country rows
# Australia block
aus_mask = (df["Country/Region"] == "Australia")
aus_pop = acc_pop_df.loc[acc_pop_df.Country == "Australia", "Population"].sum()
aus_pop_ls = [aus_pop for i in range(6)]
aus_row_ls = []
for date in np.sort(df.loc[aus_mask, "Date"].unique()):

    # start a new row by duplicating an extant row
    all_aus_agg_row = df[aus_mask & (df.Date == date)].head(1)

    # calculate vars
    agg_vars = df.loc[aus_mask & (df.Date == date), case_vars + date_vars].sum().values
    all_aus_agg_row.loc[:, case_vars + date_vars] = agg_vars
    all_aus_agg_row.loc[:, cap_vars + cap_date_vars] = agg_vars * factor / aus_pop_ls

    # tweak row to reflect that its for th whole country
    all_aus_agg_row["Province/State"] = "nan"

    # add row to list
    aus_row_ls.append(all_aus_agg_row)

# combine new rows into dataframe
all_aus_df = pd.concat(aus_row_ls, ignore_index=True)

# delete extant rows for whole country
df = df[~(aus_mask & (df["Province/State"] == "nan"))].reset_index(drop=True)

# add each new row to end of dataframe
for row in range(all_aus_df.shape[0]):
    df.loc[len(df.index)] = all_aus_df.iloc[row,:].values

# China block
chn_mask = (df["Country/Region"] == "China")
chn_pop = acc_pop_df.loc[acc_pop_df.Country == "China", "Population"].sum()
chn_pop_ls = [chn_pop for i in range(6)]
chn_row_ls = []
#print(df[chn_mask & (df["Province/State"] == "nan")])
for date in np.sort(df.loc[chn_mask, "Date"].unique()):
    
    # start a new row by duplicating an extant row
    all_chn_agg_row = df[chn_mask & (df.Date == date)].head(1)

    # calculate vars
    agg_vars = df.loc[chn_mask & (df.Date == date), case_vars + date_vars].sum().values
    all_chn_agg_row.loc[:, case_vars + date_vars] = agg_vars
    all_chn_agg_row.loc[:, cap_vars + cap_date_vars] = agg_vars * factor / chn_pop_ls
    
    # tweak row to reflect that its for th whole country
    all_chn_agg_row["Province/State"] = "nan"

    # add row to list
    chn_row_ls.append(all_chn_agg_row)

# combine new rows into dataframe
all_chn_df = pd.concat(chn_row_ls, ignore_index=True)

# delete extant rows for whole country
df = df[~(chn_mask & (df["Province/State"] == "nan"))].reset_index(drop=True)

# add each new row to end of dataframe
for row in range(all_chn_df.shape[0]):
    df.loc[len(df.index)] = all_chn_df.iloc[row,:].values

# Canada block
can_mask = (df["Country/Region"] == "Canada")
can_pop = acc_pop_df.loc[acc_pop_df.Country == "Canada", "Population"].sum()
can_pop_ls = [can_pop for i in range(6)]
can_row_ls = []
for date in np.sort(df.loc[can_mask, "Date"].unique()):

    # start a new row by duplicating an extant row
    all_can_agg_row = df[can_mask & (df.Date == date)].head(1)

    # calculate vars
    agg_vars = df.loc[can_mask & (df.Date == date), case_vars + date_vars].sum().values
    all_can_agg_row.loc[:, case_vars + date_vars] = agg_vars
    all_can_agg_row.loc[:, cap_vars + cap_date_vars] = agg_vars * factor / can_pop_ls

    # tweak row to reflect that its for th whole country
    all_can_agg_row["Province/State"] = "nan"

    # add row to list
    can_row_ls.append(all_can_agg_row)

# combine new rows into dataframe
all_can_df = pd.concat(can_row_ls, ignore_index=True)

# delete extant rows for whole country
df = df[~(can_mask & (df["Province/State"] == "nan"))].reset_index(drop=True)

# add each new row to end of dataframe
for row in range(all_can_df.shape[0]):
    df.loc[len(df.index)] = all_can_df.iloc[row,:].values

In [290]:
# calculate World country per capita variables
world_pop_df = pd.read_pickle(r"C:\Users\adiad\Anaconda3\envs\CovidApp36\covidapp\data_clean\world_pop_df.pkl")
world_pop_df.head(3)

,Country,Population
0,China,1433783686
1,India,1366417754
2,United States,329064917


In [291]:
np.sort(world_pop_df.Country.unique().tolist())

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Caribbean Netherlands', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Croatia',
       'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'DR Congo',
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic',
       'East Timor', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'F.S. Micron

In [292]:
# print countries in epidemiological data which aren't matched in the population data
[country for country in np.sort(df["Country/Region"].unique()) if country not in world_pop_df.Country.to_list()]

['Channel Islands',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Cruise Ship',
 'Guinea Bissau',
 'Hong Kong S.A.R.',
 'Jersey',
 'Kosovo',
 'Macedonia',
 'Republic of Congo',
 'Republic of Serbia',
 'Reunion',
 'Saint Barthelemy',
 'Saint Martin',
 'Sao Tome and Principe',
 'The Bahamas',
 'United Republic of Tanzania',
 'United States of America',
 'Vatican']

In [293]:
world_pop_df.loc[world_pop_df.Country == "DR Congo", "Country"] = "Republic of Congo"
world_pop_df.loc[world_pop_df.Country == "Serbia", "Country"] = "Republic of Serbia"
world_pop_df.loc[world_pop_df.Country == "Bahamas", "Country"] = "The Bahamas"
world_pop_df.loc[world_pop_df.Country == "Tanzania", "Country"] = "United Republic of Tanzania"
world_pop_df.loc[world_pop_df.Country == "Vatican City", "Country"] = "Vatican"

In [294]:
# create a dict to transform states/provinces to their population
world_pop_dict = {}
for country in world_pop_df.Country.unique():
    #print(province)
    #print(ac_pop_df[ac_pop_df.State == province])
    #print("   ")
    pop = world_pop_df.loc[world_pop_df.Country == country, "Population"].values[0]
    world_pop_dict.update({country:pop})

print("Country, Population")
for x in list(world_pop_dict)[:3]:
    print(x, ",", world_pop_dict[x])

# calculate per capita values for countries
countries = np.sort(df.loc[df["Country/Region"].isin(world_pop_df.Country), \
                           "Country/Region"].unique()).tolist()
countries.remove("Australia") # the US doesn't match so it is not in this blacklist
countries.remove("China")
world_mask = df["Country/Region"].isin(countries)

df.loc[world_mask, "Population"] = df.loc[world_mask, "Country/Region"].replace(world_pop_dict)
pop_arr = df.loc[world_mask, "Population"].values

df.loc[world_mask, cap_vars] = df.loc[world_mask, case_vars].values / \
                                      np.stack((pop_arr for i in range(3)), axis=1)
df.loc[world_mask, cap_date_vars] = df.loc[world_mask, date_vars].values * per_day_cap_factor / \
                                           np.stack((pop_arr for i in range(3)), axis=1)

Country, Population
China , 1433783686
India , 1366417754
United States , 329064917


In [295]:
# define categorical variable which indicates the intended map scope of each row
df["MapScope"] = ""
df.loc[(df["Country/Region"] == "United States of America") & \
       (df.Date > pd.datetime(2020, 3, 21)) & (df.County != "nan"), "MapScope"] = "US Counties"
df.loc[(df["Country/Region"] == "United States of America") & \
       (df["County"] == "nan") & (df["Province/State"] != "nan"), "MapScope"] = "US States"
df.loc[(df["Country/Region"] == "China") & (df["Province/State"] != "nan") & \
       (df.Date > pd.datetime(2020, 1, 22)), "MapScope"] = "China Provinces"
df.loc[(df["Country/Region"] == "Australia") & \
       (df["Province/State"] != "nan"), "MapScope"] = "Australia States"
df.loc[(df["Country/Region"] == "Canada") & (df["Province/State"] != "nan"), "MapScope"] = "Canada Provinces"
df.loc[(df["Province/State"] == "nan"), "MapScope"] = "Countries"
df = df.sort_values(by=["MapScope", "Date"])

In [296]:
def memory_usage(df):
    return(round(df.memory_usage(deep=True).sum() / 1024 ** 2, 2))

print('Memory used:', memory_usage(df), 'Mb')
print(df.dtypes)

Memory used: 488.76 Mb
Country/Region                       object
Province/State                       object
County                               object
Date                         datetime64[ns]
Confirmed                           float64
Deaths                              float64
Recovered                           float64
Latitude                            float64
Longitude                           float64
FIPS                                 object
Active                              float64
Incidence_Rate                      float64
Case-Fatality_Ratio                 float64
ConfirmedPerDate                     object
RecoveredPerDate                     object
DeathsPerDate                        object
USstateAbbr                          object
ConfirmedPerCapita                  float64
RecoveredPerCapita                  float64
DeathsPerCapita                     float64
ConfirmedPerDatePerCapita           float64
RecoveredPerDatePerCapita           float64
DeathsPer

In [297]:
# reduce the memory footprint of the dataframe
df["Country/Region"] = df["Country/Region"].astype("category")
df["Province/State"] = df["Province/State"].astype("category")
df["County"] = df["County"].astype("category")
df["MapScope"] = df["MapScope"].astype("category")

df.Confirmed = df.Confirmed.fillna(0).astype("uint32")
df.Recovered = df.Recovered.fillna(0).astype("uint32")
df.Deaths = df.Deaths.fillna(0).astype("uint32")

# PerDate values were calculated, if the quantity decreased from the
# prior day, then the PerDate value will be negative.
# Negative values will be clipped/replaced with zeros so unsigned
# integers can be used.
df.ConfirmedPerDate[df.ConfirmedPerDate < 0] = 0
df.ConfirmedPerDate = df.ConfirmedPerDate.fillna(0).round().astype("uint32")
df.RecoveredPerDate[df.RecoveredPerDate < 0] = 0
df.RecoveredPerDate = df.RecoveredPerDate.fillna(0).round().astype("uint16")
df.DeathsPerDate[df.DeathsPerDate < 0] = 0
df.DeathsPerDate = df.DeathsPerDate.fillna(0).round().astype("uint16")

df.ConfirmedPerCapita[df.ConfirmedPerCapita < 0] = 0
df.ConfirmedPerCapita = df.ConfirmedPerCapita.fillna(0).astype("float16")
df.RecoveredPerCapita[df.RecoveredPerCapita < 0] = 0
df.RecoveredPerCapita = df.RecoveredPerCapita.fillna(0).astype("float16")
df.DeathsPerCapita[df.DeathsPerCapita < 0] = 0
df.DeathsPerCapita = df.DeathsPerCapita.fillna(0).astype("float16")

df.ConfirmedPerDatePerCapita[df.ConfirmedPerDatePerCapita < 0] = 0
df.ConfirmedPerDatePerCapita = df.ConfirmedPerDatePerCapita.fillna(0)
df.RecoveredPerDatePerCapita[df.RecoveredPerDatePerCapita < 0] = 0
df.RecoveredPerDatePerCapita = df.RecoveredPerDatePerCapita.fillna(0)
df.DeathsPerDatePerCapita[df.DeathsPerDatePerCapita < 0] = 0
df.DeathsPerDatePerCapita = df.DeathsPerDatePerCapita.fillna(0)

#df.Latitude = df.Latitude.astype("float16")
#df.Longitude = df.Longitude.astype("float16")
df.FIPS = df.FIPS.astype("category")

df = df.drop(["Active", "Latitude", "Longitude", "USstateAbbr", "Population"], axis=1)

print('Memory used:', memory_usage(df), 'Mb')
print(df.dtypes)

Memory used: 62.19 Mb
Country/Region                     category
Province/State                     category
County                             category
Date                         datetime64[ns]
Confirmed                            uint32
Deaths                               uint32
Recovered                            uint32
FIPS                               category
Incidence_Rate                      float64
Case-Fatality_Ratio                 float64
ConfirmedPerDate                     uint32
RecoveredPerDate                     uint16
DeathsPerDate                        uint16
ConfirmedPerCapita                  float16
RecoveredPerCapita                  float16
DeathsPerCapita                     float16
ConfirmedPerDatePerCapita           float64
RecoveredPerDatePerCapita           float64
DeathsPerDatePerCapita              float64
MapScope                           category
dtype: object


In [298]:
# Save clean dataframe as a pickle file
pkl_file_path = r"C:\Users\adiad\Anaconda3\envs\CovidApp36\covidapp\data_clean\\"
pkl_file_name = "Johns_Hopkins_Clean.pkl"
df.to_pickle(pkl_file_path + pkl_file_name)

In [299]:
# Upload new pickle file to Google Drive
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#gauth = GoogleAuth()
#gauth.LoadClientConfigFile("C:/Users/adiad/Anaconda3/envs/CovidApp36/covidapp/secret_credentials/client_secrets.json")

#drive = GoogleDrive(gauth)

# List all my files and folders on Google Drive with their ID
#for file_list in drive.ListFile({'q': 'trashed=false'}):
#  print('Received %s files from Files.list()' % len(file_list)) # <= 10
#  for file1 in file_list:
#      print('title: %s, id: %s' % (file1['title'], file1['id']))

In [300]:
#g_file = drive.CreateFile({'id': '1KuubeQzOHAzh_TuNyK2w1XO_L8zXHTRF'})

# Read file and set it as a content of this instance.
#g_file.SetContentFile(pkl_file_path + pkl_file_name)
#g_file.Upload() # Upload the file.
#print('title: %s, mimeType: %s' % (g_file['title'], g_file['mimeType']))
# title: cat.png, mimeType: image/png

In [301]:
t1 = time.time()
(t1 - t0)/60

54.609790011247

In [302]:
df.shape[0]

717771

Pre-render heatmap animation data for US counties

In [303]:
import pickle, json
import datetime as dt
from plotly import subplots
from plotly import graph_objects as go

# converts numpy's datetime64 dtype (used by pandas) to datetime.datetime()
def numpy_dt64_to_dt(dt64):
    day_timestamp_dt = (dt64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    day_dt = dt.datetime.utcfromtimestamp(day_timestamp_dt)
    return day_dt

# get mapbox token
token = open("C:/Users/adiad/Anaconda3/envs/CovidApp36/covidapp/secret_credentials/.mapbox_token").read()

with open("C:/Users/adiad/Anaconda3/envs/CovidApp36/covidapp/data_clean/us_county_geo.json") as f:
    us_counties_json = json.load(f)

# set app.py function input variables
map_scope = "UScounties"
map_var = "Confirmed"
map_calc = "Total"
map_scale = "Logarithmic"
map_norm_type = ""
map_norm_val = 100000
init_spinner_style = "this is not None"

# determine initial date to plot
date_has_data_df = plot_df.groupby(["Date"]).sum().reset_index()
map_date = date_has_data_df.loc[date_has_data_df[plot_var] > 0, "Date"].max()

#if map_scale == "Logarithmic":
#    if map_norm_type == "PerCapita":
#        plot_df["CaseVar"] = np.log10(plot_df[plot_var]*map_norm_val)
#        
#    else:
#        plot_df["CaseVar"] = np.log10(plot_df[plot_var])
#
#else:
#    if map_norm_type == "PerCapita":
#        plot_df["CaseVar"] = plot_df[plot_var]*map_norm_val
#
#    else:
#        plot_df["CaseVar"] = plot_df[plot_var]
#
#x_arr = plot_df.CaseVar.values
#var_finite = x[(x_arr != 0) & (x_arr != -np.inf) & (x_arr != np.inf)]
#
#if len(var_finite) > 0:
#    date_has_data_df = plot_df.groupby(["Date"]).sum().reset_index()
#    map_date = date_has_data_df.loc[date_has_data_df[plot_var] > 0, "Date"].max()
#else:
#    init_days = np.sort(df[df["MapScope"] == "US Counties"].Date.unique())
#    map_date = numpy_dt64_to_dt(init_days[-1])

#### BEGIN app.py function block
# test if this is the initial execution of this callback
is_init = (init_spinner_style is None)

# only generate a new heatmap if the user initialized this callback
if is_init:
    fig = init_heatmap

    # determine valid date range for the date picker
    plot_df = df[df["MapScope"] == "US Counties"]
    days = np.sort(plot_df.Date.unique())
    picker_min_date = numpy_dt64_to_dt(days[0])
    picker_max_date = numpy_dt64_to_dt(days[-1])

else:

    # set null values of map parameters
    if map_calc == "Total":
        map_calc = ""
    if map_norm_type == "None":
        map_norm_type = ""
    plot_var = map_var + map_calc + map_norm_type

    # set variables conditioned on the map scope
    if map_scope == "UScounties":
        geo_json = us_counties_json
        plot_df = df[df["MapScope"] == "US Counties"]
        plot_df["AreaLabel"] = plot_df.County.astype(str) + ", " + plot_df["Province/State"].astype(str)
        location_var = "FIPS"
        geo_json_name_field = None
        map_center = {"lat": 37.0902, "lon": -95.7129}
        title = "US counties"
        init_zoom = 3
    
    elif map_scope == "USstates":
        geo_json = us_states_json
        plot_df = df[df["MapScope"] == "US States"]
        plot_df["AreaLabel"] = plot_df["Province/State"].astype(str)
        location_var = "Province/State"
        geo_json_name_field = "properties.NAME"
        map_center = {"lat": 37.0902, "lon": -95.7129}
        title = "US states"
        init_zoom = 3
    
    elif map_scope == "China":
        geo_json = china_json
        plot_df = df[df["MapScope"] == "China Provinces"]
        plot_df["AreaLabel"] = plot_df["Province/State"].astype(str)
        location_var = "Province/State"
        geo_json_name_field = "properties.NL_NAME_1"
        map_center = {"lat": 37.110573, "lon": 106.493924}
        title = "Chinese provinces"
        init_zoom = 2
    
    elif map_scope == "Australia":
        geo_json = australia_json
        plot_df = df[df["MapScope"] == "Australia States"]
        plot_df["AreaLabel"] = plot_df["Province/State"].astype(str)
        location_var = "Province/State"
        geo_json_name_field = None
        map_center = {"lat": -26, "lon": 133 + 25/60}
        title = "Australian states"
        init_zoom = 3
    
    elif map_scope == "Canada":
        geo_json = canada_json
        plot_df = df[df["MapScope"] == "Canada Provinces"]
        plot_df["AreaLabel"] = plot_df["Province/State"].astype(str)
        location_var = "Province/State"
        geo_json_name_field = "properties.PRENAME"
        map_center = {"lat": 58, "lon": -96 - 48/60}
        title = "Canadian Provinces"
        init_zoom = 2
    
    elif map_scope == "World":
        geo_json = world_json
        plot_df = df[df["MapScope"] == "Countries"]
        plot_df["AreaLabel"] = plot_df["Country/Region"].astype(str)
        location_var = "Country/Region"
        geo_json_name_field = "properties.ADMIN"
        map_center = {"lat": 0, "lon": 0}
        title = "Countries"
        init_zoom = 0

    # set axis variables conditioned on scale settings
    def get_min_max(x_arr):
        var_finite = x_arr[(x_arr != 0) & (x_arr != -np.inf) & (x_arr != np.inf)]
        if len(var_finite) > 0:
            var_min = min(var_finite)
            var_max = max(var_finite)
        else:
            var_min = 0
            var_max = 0
        return var_min, var_max
    
    # determine valid date range for the date picker
    days = np.sort(plot_df.Date.unique())
    picker_min_date = numpy_dt64_to_dt(days[0]) 
    picker_max_date = numpy_dt64_to_dt(days[-1])
    
    # setup scales
    log_txt = ["1e-6", "1e-5", "1e-4", ".001", ".01", ".1", \
                "1", "10", "100", "1K", "10K", "100K", "1M"]
    map_log_hvr_txt = "Cases per " + log_txt[int(np.log10(map_norm_val)) + 6] + " Capita: "
    if map_scale == "Logarithmic":
        bar_scale_type = "log"
        map_tick_mode = "array"
        map_tick_vals = np.arange(-6, 7)
        map_tick_txt = log_txt
        
        if map_norm_type == "PerCapita":
            plot_df["CaseVar"] = np.log10(plot_df[plot_var]*map_norm_val)
            
        else:
            plot_df["CaseVar"] = np.log10(plot_df[plot_var])
        
        var_min, var_max = get_min_max(plot_df.CaseVar.values)
        plot_range = np.array([var_min, var_max])
    
    else:
        bar_scale_type = "linear"
        map_tick_mode = "auto"
        map_tick_vals = []
        map_tick_txt = []

        if map_norm_type == "PerCapita":
            plot_df["CaseVar"] = plot_df[plot_var]*map_norm_val

        else:
            plot_df["CaseVar"] = plot_df[plot_var]
        
        var_min, var_max = get_min_max(plot_df.CaseVar.values)
        plot_range = np.array([0, var_max])
    
    if map_var == "Recovered":
        heat_color_scale = "ylgn"
        bar_color = "rgb(69, 161, 69)"
    else:
        heat_color_scale = "ylorrd"
        bar_color = "rgb(236, 62, 19)"

    # limit remaining calcs to data pertaining to picked date
    plot_day_df = plot_df[plot_df.Date == map_date]

    # define custom hover data
    cust_data = np.dstack((plot_day_df.loc[:, map_var + map_calc].values, \
                        plot_day_df.loc[:, map_var + map_calc + "PerCapita"]. \
                                    values*map_norm_val))[0]
    location_series = plot_day_df[location_var]
    if map_norm_type == "PerCapita":
        bar_txt_format = "{:.2e}"
    else:
        bar_txt_format = "{:,.0f}"
    
    # define the left bar plot
    bar_df = plot_day_df.nlargest(10, "CaseVar", keep="all").reset_index()
    bar_df = bar_df.head(10) # nlargest may return more than 10 rows if there are duplicate values
    bar_df = bar_df[bar_df.CaseVar > -np.inf]
    nrows = bar_df.shape[0]
    bar_df = bar_df.iloc[np.arange(nrows - 1, -1, -1),:] # reverse order of top 10 rows

    # plotly does not tolerate changing the number of bars in 
    # a bar graph during animation define a function to pad 
    # data arrays with blank elements so the bar graph always 
    # has 10 elements
    def pad_10_arr(x, pad_val, unique_fill_bool):
        xlen = len(x)
        if xlen == 10:
            result = x
        else:
            npad = 10 - xlen
            fill_arr = np.array([pad_val for i in range(npad)])

            # shorten each string fill element in array to make the elements unique
            if unique_fill_bool:
                fill_arr = [item[i:] for i, item in enumerate(fill_arr)]
            
            result = np.append(fill_arr, x)
        return result

    # only build the bar plot if there is data to plot
    if plot_day_df[plot_var].max() > 0:
        no_data = False

        max_width_label = 25
        if map_scope == "UScounties":

            # some of the county, state labels are too long, taking up too much space
            # in the figure.  Long labels will have the county label trimmed with an ellipsis appended.
            labels_to_trim = bar_df["AreaLabel"].astype(str).str.len() > max_width_label
            county_len_arr = max_width_label - 5 - bar_df.loc[labels_to_trim, "Province/State"].astype(str).str.len().values
            county_abbr = [bar_df.loc[labels_to_trim, "County"].astype(str).values[i][:county_len_arr[i]] \
                        for i in range(len(county_len_arr))]
            state_abbr = bar_df.loc[labels_to_trim, "Province/State"].astype(str).values.tolist()
            county_state_abbr = [county_abbr[i] + "..., " + state_abbr[i] for i in range(len(county_abbr))]
            bar_df.loc[labels_to_trim, "AreaLabel"] = county_state_abbr
        elif map_scope == "Australia":

            # only one label needs to be trimmed
            long_label = "Australian Capital Territory"
            labels_to_trim = bar_df["AreaLabel"].astype(str) == long_label
            bar_df.loc[labels_to_trim, "AreaLabel"] = long_label[:(max_width_label - 3)] + "..."

        # bar labels must be padded so all labels have the same length
        # as some labels disappear and others are introduced,
        # varied-length label cause bad animation behavior
        area_labels = [label.rjust(max_width_label) for label in bar_df.AreaLabel.values]
        
        if map_scale == "Logarithmic":
            bar_df["CaseVarPlot"] = np.power(np.ones(10)*10, bar_df.CaseVar.values)
        else:
            bar_df["CaseVarPlot"] = bar_df.CaseVar
        bar_df["ValLabels"] = bar_df.CaseVarPlot.astype("float")
        bar_fig_data = go.Bar(x=pad_10_arr(bar_df.CaseVarPlot.values, 0, False),
                                y=pad_10_arr(area_labels, " " * max_width_label, True),
                                text=pad_10_arr(bar_df.ValLabels.map(bar_txt_format.format).values, "", False),
                                textposition="auto",
                                hoverinfo="none",
                                orientation="h",
                                marker_color=bar_color,
                                name="")
    else:
        no_data = True
        bar_fig_data = go.Bar(x=[],
                                y=[],
                                orientation="h",
                                name="")
    
    # build the heatmap
    heat_fig_data =go.Choroplethmapbox(geojson=geo_json,
                                        locations=location_series,
                                        featureidkey=geo_json_name_field,
                                        z=plot_day_df.CaseVar,
                                        zmin=plot_range[0], # min([plot_df.CaseVar.min(), 0]),
                                        zmax=plot_range[1], # plot_df.CaseVar.max(),
                                        customdata=cust_data,
                                        name="",
                                        text=plot_day_df.AreaLabel,
                                        hovertemplate="<b>%{text}</b><br>" + \
                                                        "<b>Cases</b>: %{customdata[0]:,}<br>" + \
                                                        "<b>" + map_log_hvr_txt + "</b>: %{customdata[1]:.2e}",
                                        colorbar=dict(outlinewidth=1,
                                                        outlinecolor="#333333",
                                                        len=0.9,
                                                        lenmode="fraction",
                                                        xpad=30,
                                                        xanchor="right",
                                                        bgcolor=None,
                                                        title=dict(text="Cases",
                                                                font=dict(size=14)),
                                                        tickmode=map_tick_mode,
                                                        tickvals=map_tick_vals,
                                                        ticktext=map_tick_txt,
                                                        tickcolor="#333333",
                                                        tickwidth=2,
                                                        tickfont=dict(color="#333333",
                                                                    size=12)),
                                        colorscale=heat_color_scale,
                                        marker_opacity=0.7,
                                        marker_line_width=0)

    ## define animation controls
    #frame_dur = 1000 # milliseconds, controls animation speed
    #fig_ctrls = []
    #sliders_dict = dict()
    #
    ## only define the animation controls of there is data to plot
    #if plot_df[plot_var].max() > 0:
    #    fig_ctrls = [dict(type="buttons",
    #                      buttons=[dict(label="Play",
    #                                    method="animate",
    #                                    args=[None,
    #                                        dict(frame=dict(duration=frame_dur,
    #                                                        redraw=True),
    #                                             fromcurrent=True)]),
    #                            dict(label="Pause",
    #                                 method="animate",
    #                                 args=[[None],
    #                                       dict(frame=dict(duration=0,
    #                                                       redraw=True),
    #                                            mode="immediate")])],
    #                      direction="left",
    #                      pad={"r": 10, "t": 35},
    #                      showactive=False,
    #                      x=0.1,
    #                      xanchor="right",
    #                      y=0,
    #                      yanchor="top")]
    #
    #    if (not is_init):
    #        sliders_dict = dict(active=init_date_ind,
    #                            visible=True,
    #                            yanchor="top",
    #                            xanchor="left",
    #                            currentvalue=dict(font=dict(size=14),
    #                                              prefix="Plotted Date: ",
    #                                              visible=True,
    #                                              xanchor="center"),
    #                            pad=dict(b=10,
    #                                     t=10),
    #                            len=0.875,
    #                            x=0.125,
    #                            y=0,
    #                            steps=[])
    # 
    ##toc_a = time.perf_counter()
    #
    ## define the animation frames
    #fig_frames = []
    #
    #for day in days:
    #
    #    # this code repeating what was done to build the initial bar plot above
    #    # .query() method provides faster filtering
    #    plot_day_df = plot_df.query("Date == @day") #plot_day_df = plot_df[plot_df.Date == day]
    #    bar_df = plot_day_df.nlargest(10, "CaseVar", keep="all").reset_index()
    #    bar_df = bar_df.head(10) # nlargest may return more than 10 rows if there are duplicate values
    #    INF = np.inf
    #    bar_df = bar_df.query("CaseVar > - @INF") #bar_df = bar_df[bar_df.CaseVar > -np.inf]
    #    nrows = bar_df.shape[0]
    #    bar_df = bar_df.iloc[np.arange(nrows - 1, -1, -1),:] # reverse order of top 10 rows
    #    if map_scope == "UScounties":
    #        labels_to_trim = bar_df["AreaLabel"].astype(str).str.len() > max_width_label
    #        county_len_arr = max_width_label - 5 - bar_df.loc[labels_to_trim, "Province/State"].astype(str).str.len().values
    #        county_abbr = [bar_df.loc[labels_to_trim, "County"].astype(str).values[i][:county_len_arr[i]] \
    #                    for i in range(len(county_len_arr))]
    #        state_abbr = bar_df.loc[labels_to_trim, "Province/State"].astype(str).values.tolist()
    #        county_state_abbr = [county_abbr[i] + "..., " + state_abbr[i] for i in range(len(county_abbr))]
    #        bar_df.loc[labels_to_trim, "AreaLabel"] = county_state_abbr
    #    elif map_scope == "Australia":
    #        long_label = "Australian Capital Territory"
    #        labels_to_trim = bar_df["AreaLabel"].astype(str) == long_label
    #        bar_df.loc[labels_to_trim, "AreaLabel"] = long_label[:(max_width_label - 3)] + "..."
    #    area_labels = [label.rjust(max_width_label) for label in bar_df.AreaLabel.values]
    #    bar_df["ValLabels"] = bar_df.CaseVar.astype("float")
    #
    #    # this code repeats what was done to build the initial heatmap above
    #    cust_data = np.dstack((plot_day_df.loc[:, map_var + map_calc].values, \
    #                        plot_day_df.loc[:, map_var + map_calc + "PerCapita"]. \
    #                                    values*map_norm_val))[0]
    #    location_series = plot_day_df[location_var]
    #    
    #    # define the frame, repeating what was done for the initial plots above
    #    frame = go.Frame(data=[go.Bar(x=pad_10_arr(bar_df[plot_var].values, 0, False),
    #                                    y=pad_10_arr(area_labels, " " * max_width_label, True),
    #                                    text=pad_10_arr(bar_df.ValLabels.map(bar_txt_format.format). \
    #                                                            values, "", False),
    #                                    textposition="auto",
    #                                    hoverinfo="none",
    #                                    name=""),
    #                            go.Choroplethmapbox(locations=location_series,
    #                                                featureidkey=geo_json_name_field,
    #                                                z=plot_day_df.CaseVar,
    #                                                customdata=cust_data,
    #                                                name="",
    #                                                text=plot_day_df.AreaLabel,
    #                                                hovertemplate="<b>%{text}</b><br>" + \
    #                                                                "<b>Cases</b>: %{customdata[0]:,}<br>" + \
    #                                                                "<b>" + map_log_hvr_txt + "</b>: %{customdata[1]:.2e}")],
    #                        name=numpy_dt64_to_str(day))
    #    fig_frames.append(frame)
    #
    #    # define the slider step
    #    slider_step = dict(args=[[numpy_dt64_to_str(day)],
    #                                dict(mode="immediate",
    #                                    frame=dict(duration=300,
    #                                                redraw=True))],
    #                        method="animate",
    #                        label=numpy_dt64_to_str(day))
    #    sliders_dict["steps"].append(slider_step)
    #
    ##toc_b = time.perf_counter()

    # Assemble the entire figure based on the components defined above
    fig = subplots.make_subplots(rows=1, cols=2, column_widths=[0.2, 0.8],
                                    subplot_titles=("Top 10 " + title, ""),
                                    horizontal_spacing=0.05,
                                    specs=[[{"type": "bar"},
                                            {"type": "choroplethmapbox"}]])
    fig.add_trace(bar_fig_data, row=1, col=1)
    fig.add_trace(heat_fig_data, row=1, col=2)
    fig.update_layout(mapbox_style="light",
                        mapbox_zoom=init_zoom,
                        mapbox_accesstoken=token,
                        mapbox_center=map_center,
                        margin={"r": 10, "t": 20, "l": 10, "b": 10},
                        plot_bgcolor="white")
                        #sliders=[sliders_dict],
                        #updatemenus=fig_ctrls)
    #fig["frames"] = fig_frames
    
    # update the bar plot axes
    if no_data:
        fig.update_xaxes(showticklabels=False)
        fig.update_yaxes(showticklabels=False)
    else:
        fig.update_xaxes(type=bar_scale_type,
                            ticks="outside",
                            range=plot_range,
                            showgrid=True,
                            gridwidth=0.5,
                            gridcolor="#CCCCCC")
        fig.update_yaxes(tickfont=dict(family="Courier New, monospace",
                                        size=13))

    if no_data:
        # add annotation when theres no data explaining as such
        fig["layout"]["annotations"] = [dict(x=0,
                                                y=0,
                                                xref="x1", 
                                                yref="y1",
                                                text="All<br>" + title + "<br>have reported<br>zero " + \
                                                    map_var + "<br>cases to date",
                                                showarrow=False,
                                                font=dict(size=16))]
    else:
        # modify the bar plot title font properties
        fig["layout"]["annotations"][0]["font"] = dict(size=16)
### END app.py function block

# write the finished data structures as pickle files
with open("C:/Users/adiad/Anaconda3/envs/CovidApp36/covidapp/data_clean/init_heatmap.pkl", "wb") as handle:
    pickle.dump(fig, handle, protocol=pickle.HIGHEST_PROTOCOL)

C:\Users\adiad\Anaconda3\envs\CovidApp36\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future



In [304]:
import boto3
from botocore.config import Config

bucket_name = "my-covid-data-7918"

# uploading new dataframe to s3
content = open(pkl_file_path + pkl_file_name, "rb")
s3 = boto3.client("s3")
s3.put_object(
   Bucket = bucket_name,
   Key = pkl_file_name,
   StorageClass = "STANDARD_IA",
   Body = content
)

# uploading new initial heatmap data object to s3
content = open("C:/Users/adiad/Anaconda3/envs/CovidApp36/covidapp/data_clean/init_heatmap.pkl", "rb")
s3.put_object(
   Bucket = bucket_name,
   Key = "init_heatmap.pkl",
   StorageClass = "STANDARD_IA",
   Body = content
)

{'ResponseMetadata': {'RequestId': '17CECC915B159807',
  'HostId': 'YJFGaFT132PIjLiYjdZwMI6hjzoSOC663ZvwGpBChOdOOfrN0UsXW2YVqc8TFSpcnzDiqf0mXKw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'YJFGaFT132PIjLiYjdZwMI6hjzoSOC663ZvwGpBChOdOOfrN0UsXW2YVqc8TFSpcnzDiqf0mXKw=',
   'x-amz-request-id': '17CECC915B159807',
   'date': 'Mon, 10 Aug 2020 22:37:26 GMT',
   'etag': '"4b9f7c4727b72729333e159750682fdc"',
   'x-amz-storage-class': 'STANDARD_IA',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4b9f7c4727b72729333e159750682fdc"'}

In [305]:
# restart my web app, forcing it to download the new data
# from https://aws.amazon.com/premiumsupport/knowledge-center/schedule-elastic-beanstalk-stop-restart/

eb_config = Config(
    region_name = 'us-east-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)
client = boto3.client('elasticbeanstalk', config=eb_config)

envid=['e-8rvgrz5iqy']

try:
    for appid in range(len(envid)):
        response = client.restart_app_server(EnvironmentId=envid[appid])
        if response:
            print('Restore environment %s' %str(envid[appid]))
        else:
            print('Failed to Restore environment %s' %str(envid[appid]))

except Exception as e:
    print(e)

Restore environment e-8rvgrz5iqy


In [306]:
perc_df = df[(df["Country/Region"] == "United States of America") & (df.County != "nan") & (df.Date == pd.datetime(2020, 6, 30))]
perc_df["Percentile"] = perc_df["ConfirmedPerCapita"].rank(pct=True)
perc_df.sort_values("Percentile", ascending=False)

,Country/Region,Province/State,County,Date,Confirmed,Deaths,Recovered,FIPS,Incidence_Rate,Case-Fatality_Ratio,...,RecoveredPerDate,DeathsPerDate,ConfirmedPerCapita,RecoveredPerCapita,DeathsPerCapita,ConfirmedPerDatePerCapita,RecoveredPerDatePerCapita,DeathsPerDatePerCapita,MapScope,Percentile
288486,United States of America,New York,New York,2020-06-30,215179,23096,0,36061,2581.069010,10.733389,...,0,626,0.132080,0.0,0.014183,0.000147,0.0,0.000384,US Counties,1.000000
379925,United States of America,Tennessee,Trousdale,2020-06-30,1485,5,0,47169,13160.226870,0.336700,...,0,0,0.131592,0.0,0.000443,0.000266,0.0,0.000000,US Counties,0.999693
374620,United States of America,Tennessee,Lake,2020-06-30,690,0,0,47095,9834.663626,0.000000,...,0,0,0.098328,0.0,0.000000,0.000000,0.0,0.000000,US Counties,0.999386
264538,United States of America,Nebraska,Dakota,2020-06-30,1759,31,0,31043,8783.581344,1.762365,...,0,0,0.087830,0.0,0.001548,0.000000,0.0,0.000000,US Counties,0.999078
148527,United States of America,Iowa,Buena Vista,2020-06-30,1695,11,0,19021,8639.143731,0.648968,...,0,0,0.086365,0.0,0.000561,0.000051,0.0,0.000000,US Counties,0.998771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595915,United States of America,Montana,Blaine,2020-06-30,0,0,0,0,NaN,NaN,...,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,US Counties,0.036559
595916,United States of America,Montana,Carter,2020-06-30,0,0,0,0,NaN,NaN,...,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,US Counties,0.036559
595917,United States of America,Montana,Chouteau,2020-06-30,0,0,0,0,NaN,NaN,...,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,US Counties,0.036559
595918,United States of America,Montana,Daniels,2020-06-30,0,0,0,0,NaN,NaN,...,0,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,US Counties,0.036559


In [307]:
perc_df[perc_df.County == "Marion"]

,Country/Region,Province/State,County,Date,Confirmed,Deaths,Recovered,FIPS,Incidence_Rate,Case-Fatality_Ratio,...,RecoveredPerDate,DeathsPerDate,ConfirmedPerCapita,RecoveredPerCapita,DeathsPerCapita,ConfirmedPerDatePerCapita,RecoveredPerDatePerCapita,DeathsPerDatePerCapita,MapScope,Percentile
44350,United States of America,Alabama,Marion,2020-06-30,200,12,0,01093,673.196674,6.000000,...,0,0,0.006733,0.0,0.000404,0.000101,0.0,0.000000,US Counties,0.791244
58457,United States of America,Arkansas,Marion,2020-06-30,6,0,0,05089,35.941057,0.000000,...,0,0,0.000359,0.0,0.000000,0.000120,0.0,0.000000,US Counties,0.107220
87113,United States of America,Florida,Marion,2020-06-30,672,10,0,12083,183.817998,1.488095,...,0,0,0.001838,0.0,0.000027,0.000047,0.0,0.000000,US Counties,0.402151
104372,United States of America,Georgia,Marion,2020-06-30,79,2,0,13197,945.089125,2.531646,...,0,0,0.009453,0.0,0.000239,0.000120,0.0,0.000000,US Counties,0.867896
127792,United States of America,Illinois,Marion,2020-06-30,65,0,0,17121,174.707701,0.000000,...,0,0,0.001747,0.0,0.000000,0.000000,0.0,0.000000,US Counties,0.387711
140872,United States of America,Indiana,Marion,2020-06-30,11329,679,0,18097,1174.498384,5.993468,...,0,2,0.011742,0.0,0.000704,0.000052,0.0,0.000002,US Counties,0.905684
155468,United States of America,Iowa,Marion,2020-06-30,66,0,0,19125,198.478333,0.000000,...,0,0,0.001986,0.0,0.000000,0.000090,0.0,0.000000,US Counties,0.421966
167314,United States of America,Kansas,Marion,2020-06-30,9,1,0,20115,75.732077,11.111111,...,0,0,0.000757,0.0,0.000084,0.000000,0.0,0.000000,US Counties,0.197849
183041,United States of America,Kentucky,Marion,2020-06-30,51,0,0,21155,264.618897,0.000000,...,0,0,0.002645,0.0,0.000000,0.000000,0.0,0.000000,US Counties,0.511521
236365,United States of America,Mississippi,Marion,2020-06-30,240,11,0,28091,976.681724,4.583333,...,0,0,0.009766,0.0,0.000448,0.000285,0.0,0.000000,US Counties,0.875883
